In [1]:
#Librerias
import pandas as pd
import numpy as ny
import googlemaps
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut, GeocoderInsufficientPrivileges
from geopy.extra.rate_limiter import RateLimiter
import logging
import pandas as pd
import time
import re
import csv
import os
import hashlib

In [78]:
# 1. Cargamos los datos
file_path = r'H:/Nueva carpeta/overpass-turbo.eu/Parques de diverciones Centros comerciales 2014.csv'  
data = pd.read_csv(file_path, delimiter='\t')  

# 2. Realizamos una revisión general
print("Dimensiones del dataset:", data.shape)
print("\nNombres de las columnas:", data.columns.tolist())
print("\nPrimeras filas del dataset:")
print(data.head())

# 3. Mostramos la calidad de los datos
print("\nValores nulos por columna:")
print(data.isnull().sum())

print("\nTipos de datos:")
print(data.dtypes)

# 4. Realisaos una estadísticas descriptivas
print("\nEstadísticas descriptivas para columnas numéricas:")
print(data.describe())

print("\nValores únicos por columna:")
print(data.nunique())





Dimensiones del dataset: (362, 7)

Nombres de las columnas: ['name', 'addr:street', 'addr:city', 'addr:state', 'amenity', '@lat', '@lon']

Primeras filas del dataset:
                       name              addr:street      addr:city  \
0           Titusville Mall                      NaN            NaN   
1  Landmark Shopping Center                      NaN            NaN   
2            LandMark Plaza                      NaN            NaN   
3  Ellenton Premium Outlets  Factory Shops Boulevard       Ellenton   
4                  CocoWalk             Grand Avenue  Coconut Grove   

  addr:state  amenity       @lat       @lon  
0        NaN      NaN  28.575472 -80.802640  
1        NaN      NaN  28.068730 -82.436298  
2        NaN      NaN  28.068711 -82.437068  
3        NaN      NaN  27.534335 -82.506753  
4        NaN      NaN  25.728644 -80.242038  

Valores nulos por columna:
name           169
addr:street    261
addr:city      268
addr:state     283
amenity        362
@lat   

Descargamos las direcciones en base ala latitud y longitud

In [ ]:
# Configuramos el geolocalizador con un user_agent único
geolocator = Nominatim(user_agent="mi_proyecto_geocodificacion_v1")

# Función para realizar la geocodificación inversa y buscar el nombre del lugar
def reverse_geocode(lat, lon):
    try:
        location = geolocator.reverse((lat, lon), timeout=10)
        if location:
            address = location.address
            city = location.raw.get('address', {}).get('city', None)
            name = location.raw.get('name', None) or location.raw.get('address', {}).get('attraction', None)
            return address, city, name
        else:
            return "Dirección no encontrada", None, None
    except GeocoderTimedOut:
        return "Tiempo de espera agotado", None, None
    except GeocoderInsufficientPrivileges:
        return "Bloqueado: Verificar API o User-Agent", None, None

# Limpieza inicial del archivo
input_file = "H:/Nueva carpeta/overpass-turbo.eu/Parques de diverciones Centros comerciales 2014.csv"
cleaned_file = "H:/Nueva carpeta/overpass-turbo.eu/Parques de diverciones Centros comerciales 20161.csv"

with open(input_file, 'r', encoding='utf-8') as infile, open(cleaned_file, 'w', encoding='utf-8') as outfile:
    for line in infile:
        if line.strip():  # Omitir líneas vacías
            outfile.write(line)

# Cargamos el archivo limpio
try:
    data = pd.read_csv(cleaned_file, sep='\t', encoding='utf-8')
except pd.errors.ParserError as e:
    print(f"Error al cargar el archivo: {e}")
    data = pd.read_csv(cleaned_file, sep=',', encoding='utf-8', on_bad_lines='skip')

# Añadimos  la columna 'name' si no existe
if 'name' not in data.columns:
    data['name'] = None

# Actualizamos las columnas addr:street, addr:city y name
def update_address(row):
    if pd.notna(row['@lat']) and pd.notna(row['@lon']):
        address, city, name = reverse_geocode(row['@lat'], row['@lon'])
        row['addr:street'] = address
        row['addr:city'] = city
        row['name'] = name
    return row

# Aplicamos la función fila por fila con pausas para respetar los límites de solicitudes
for index, row in data.iterrows():
    data.loc[index] = update_address(row)
    time.sleep(2)

# Guardamos los resultados
output_file = "H:/Nueva carpeta/overpass-turbo.eu/Parques_direcciones_actualizadas_2014.csv"
data.to_csv(output_file, index=False)
print(data.head())




                       name  \
0           Titusville Mall   
1  Landmark Shopping Center   
2            LandMark Plaza   
3                      None   
4                  CocoWalk   

                                         addr:street   addr:city addr:state  \
0  Titusville Mall, Narvaez Drive, Titusville, Br...  Titusville        NaN   
1  Landmark Shopping Center, North 20th Street, H...        None        NaN   
2  LandMark Plaza, East Fletcher Avenue, Hillsbor...        None        NaN   
3      Manatee County, Florida, 34222, United States        None        NaN   
4  CocoWalk, 3015, Grand Avenue, Dinner Key, Miam...       Miami        NaN   

   amenity       @lat       @lon  
0      NaN  28.575472 -80.802640  
1      NaN  28.068730 -82.436298  
2      NaN  28.068711 -82.437068  
3      NaN  27.534335 -82.506753  
4      NaN  25.728644 -80.242038  


Eliminamos colunas innecesarias y cambiamos los nombres

In [ ]:
# Cargamos  los datos 
archivo = "H:/Nueva carpeta/overpass-turbo.eu/Parques_direcciones_actualizadas_2014.csv"  
data = pd.read_csv(archivo)

# Renombramos  las columnas
data.rename(columns={
    'addr:street': 'direccion',
    'amenity': 'tipo',
    '@lat': 'latitud',
    '@lon': 'longitud'
}, inplace=True)

# Dividimos la columna direccion para extraer los datos
direccion_split = data['direccion'].fillna('').str.split(',', expand=True)

# Creamos las nuebas colunas a usar
data['nombre'] = direccion_split[0].str.strip()

# Extraemos la direccion combinando todas las partes de la dirección hasta el estado
data['direccion'] = direccion_split.apply(
    lambda row: ', '.join(filter(None, [row[1].strip(), row[2].strip()])), axis=1
)

# Extraemos la ciudad siempre y cuando tenga la palabra County 
data['ciudad'] = direccion_split.apply(
    lambda row: next((x.strip() for x in row if 'County' in str(x)), 'No definido'), axis=1
)

# Extraemos el estado  "Florida"
data['estado'] = direccion_split.apply(lambda row: 'Florida', axis=1)

# Extraemos  y validamos los códigos postales 
data['codigo_postal'] = direccion_split.apply(
    lambda row: next((re.search(r'\b\d{5}\b', str(x)).group(0) for x in row if re.search(r'\b\d{5}\b', str(x))), 'No definido'), axis=1
)

# Separamos el peis "United States"
data['pais'] = direccion_split.apply(lambda row: 'United States', axis=1)

# Reordenamos las columnas 
data = data[['nombre', 'direccion', 'ciudad', 'estado', 'pais', 'codigo_postal', 'longitud', 'latitud']]

# Eliminamos  duplicados
data.drop_duplicates(inplace=True)

# Guardamos el resultado
processed_file_path = 'H:/Nueva carpeta/overpass-turbo.eu/Parques_de_diversiones_Centros_comerciales_2014.csv'
data.to_csv(processed_file_path, index=False)

# Análisamos de datos vacíos y duplicados
vacios = data.isnull().sum()
duplicados = data.duplicated().sum()

print("Análisis de datos:")
print("Valores vacíos por columna:")
print(vacios)
print(f"Número de filas duplicadas: {duplicados}")
print(data.head())


Análisis de datos:
Valores vacíos por columna:
nombre           0
direccion        0
ciudad           0
estado           0
pais             0
codigo_postal    0
longitud         0
latitud          0
dtype: int64
Número de filas duplicadas: 0
                     nombre                                  direccion  \
0           Titusville Mall                  Narvaez Drive, Titusville   
1  Landmark Shopping Center     North 20th Street, Hillsborough County   
2            LandMark Plaza  East Fletcher Avenue, Hillsborough County   
3            Manatee County                             Florida, 34222   
4                  CocoWalk                         3015, Grand Avenue   

                ciudad   estado           pais codigo_postal   longitud  \
0       Brevard County  Florida  United States         32780 -80.802640   
1  Hillsborough County  Florida  United States         33613 -82.436298   
2  Hillsborough County  Florida  United States         33613 -82.437068   
3       Manat

Eliminamos la palabra Couty

In [81]:
# Eliminamos la palabra 'County' de las columnas 'direccion' y 'ciudad'
data['direccion'] = data['direccion'].str.replace(r'\bCounty\b', '', regex=True).str.strip()
data['ciudad'] = data['ciudad'].str.replace(r'\bCounty\b', '', regex=True).str.strip()
print(data[['direccion', 'ciudad']].head())

# Guardamos el resultado 
processed_file_path = 'H:/Nueva carpeta/overpass-turbo.eu/Parques_de_diversiones_Centros_comerciales_2014.csv'
data.to_csv(processed_file_path, index=False)

                            direccion        ciudad
0           Narvaez Drive, Titusville       Brevard
1     North 20th Street, Hillsborough  Hillsborough
2  East Fletcher Avenue, Hillsborough  Hillsborough
3                      Florida, 34222       Manatee
4                  3015, Grand Avenue    Miami-Dade


Creamos las ids para nombre y ciudad

In [ ]:
# Cargamos las rutas de los archivos
input_path = r"H:/Nueva carpeta/overpass-turbo.eu/Parques_de_diversiones_Centros_comerciales_2014.csv"
output_path = r"H:/Nueva carpeta/overpass-turbo.eu/Parques_de_diversiones_Centros_comerciales_2014_final.csv"
nombres_dim_path = "H:/Nueva carpeta/overpass-turbo.eu/nombres_dim.csv"
ciudades_dim_path = "H:/Nueva carpeta/overpass-turbo.eu/ciudades_dim.csv"

# Cargamos los catálogos
def load_catalog(path, key, value):
    catalog = {}
    if os.path.exists(path):
        with open(path, mode='r', encoding='utf-8') as f:
            reader = csv.DictReader(f)
            for row in reader:
                catalog[row[key]] = int(row[value])
    return catalog

# Guardamos los catálogos de ciudad y nombres
def save_catalog(path, catalog, key, value):
    with open(path, mode='w', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=[value, key])
        writer.writeheader()
        for k, v in catalog.items():
            writer.writerow({value: v, key: k})

# Generamos IDs únicos
def get_id(value, catalog):
    if value not in catalog:
        catalog[value] = len(catalog) + 1
    return catalog[value]

# Cargamos catálogos existentes
nombres_dict = load_catalog(nombres_dim_path, "nombre", "id_nombre")
ciudades_dict = load_catalog(ciudades_dim_path, "ciudad", "id_ciudad")

# Procesamos archivo de entrada
def process_file(input_path, output_path):
    if not os.path.exists(input_path):
        raise FileNotFoundError(f"El archivo de entrada no existe: {input_path}")

    with open(input_path, mode='r', encoding='utf-8') as infile:
        reader = csv.DictReader(infile)
        original_fieldnames = reader.fieldnames

        # Validamos las columnas columnas requeridas
        required_cols = {"nombre", "direccion", "latitud", "longitud", "ciudad", "estado", "pais", "codigo_postal"}
        missing_cols = required_cols - set(original_fieldnames)
        if missing_cols:
            raise ValueError(f"El CSV no contiene las siguientes columnas requeridas: {', '.join(missing_cols)}")

        # Nombres de las columnas
        fieldnames = [
            "id_nombre", "nombre", "direccion", "latitud", "longitud",
            "id_ciudad", "ciudad", "estado", "pais", "codigo_postal"
        ]

        # ordenamos y creamos el archivo de salida
        with open(output_path, mode='w', newline='', encoding='utf-8') as outfile:
            writer = csv.DictWriter(outfile, fieldnames=fieldnames)
            writer.writeheader()

            for row in reader:
                id_nombre = get_id(row["nombre"], nombres_dict)
                id_ciudad = get_id(row["ciudad"], ciudades_dict)

                new_row = {
                    "id_nombre": id_nombre,
                    "nombre": row["nombre"],
                    "direccion": row["direccion"],
                    "latitud": row["latitud"],
                    "longitud": row["longitud"],
                    "id_ciudad": id_ciudad,
                    "ciudad": row["ciudad"],
                    "estado": row["estado"],
                    "pais": row["pais"],
                    "codigo_postal": row["codigo_postal"]
                }
                writer.writerow(new_row)

process_file(input_path, output_path)

# Guardamos el catálogos actualizados 
save_catalog(nombres_dim_path, nombres_dict, "nombre", "id_nombre")
save_catalog(ciudades_dim_path, ciudades_dict, "ciudad", "id_ciudad")

print("Proceso completado con éxito.")

Proceso completado con éxito.


Para el area de ML se combierte el archivo a formato .parquet

In [83]:
# Cargamos el archivo CSV 
csv_path = r"H:\Nueva carpeta\overpass-turbo.eu\Parques_de_diversiones_Centros_comerciales_2014_final.csv"

# Ruta del archivo Parquet de salida
parquet_path = r"H:\Nueva carpeta\overpass-turbo.eu\Parques_de_diversiones_Centros_comerciales_2014_final.parquet"

df = pd.read_csv(csv_path)

# Guardamos el archivo Parquet
df.to_parquet(parquet_path, engine='pyarrow', index=False)

print(f"Archivo convertido exitosamente a Parquet: {parquet_path}")


Archivo convertido exitosamente a Parquet: H:\Nueva carpeta\overpass-turbo.eu\Parques_de_diversiones_Centros_comerciales_2014_final.parquet


Parques de diverciones Centros comerciales 2015

In [84]:
# 1. Cargamos los datos
file_path = r'H:/Nueva carpeta/overpass-turbo.eu/Parques de diverciones Centros comerciales 2015.csv'  
data = pd.read_csv(file_path, delimiter='\t')  

# 2. Realizamos una revisión general
print("Dimensiones del dataset:", data.shape)
print("\nNombres de las columnas:", data.columns.tolist())
print("\nPrimeras filas del dataset:")
print(data.head())

# 3. Mostramos la calidad de los datos
print("\nValores nulos por columna:")
print(data.isnull().sum())

print("\nTipos de datos:")
print(data.dtypes)

# 4. Realisaos una estadísticas descriptivas
print("\nEstadísticas descriptivas para columnas numéricas:")
print(data.describe())

print("\nValores únicos por columna:")
print(data.nunique())





Dimensiones del dataset: (362, 7)

Nombres de las columnas: ['name', 'addr:street', 'addr:city', 'addr:state', 'amenity', '@lat', '@lon']

Primeras filas del dataset:
                       name              addr:street      addr:city  \
0           Titusville Mall                      NaN            NaN   
1  Landmark Shopping Center                      NaN            NaN   
2            LandMark Plaza                      NaN            NaN   
3  Ellenton Premium Outlets  Factory Shops Boulevard       Ellenton   
4                  CocoWalk             Grand Avenue  Coconut Grove   

  addr:state  amenity       @lat       @lon  
0        NaN      NaN  28.575472 -80.802640  
1        NaN      NaN  28.068730 -82.436298  
2        NaN      NaN  28.068711 -82.437068  
3        NaN      NaN  27.534335 -82.506753  
4        NaN      NaN  25.728644 -80.242038  

Valores nulos por columna:
name           169
addr:street    261
addr:city      268
addr:state     283
amenity        362
@lat   

Descargamos las direcciones en base ala latitud y longitud

In [ ]:
# Configuramos el geolocalizador con un user_agent único
geolocator = Nominatim(user_agent="mi_proyecto_geocodificacion_v1")

# Función para realizar la geocodificación inversa y buscar el nombre del lugar
def reverse_geocode(lat, lon):
    try:
        location = geolocator.reverse((lat, lon), timeout=10)
        if location:
            address = location.address
            city = location.raw.get('address', {}).get('city', None)
            name = location.raw.get('name', None) or location.raw.get('address', {}).get('attraction', None)
            return address, city, name
        else:
            return "Dirección no encontrada", None, None
    except GeocoderTimedOut:
        return "Tiempo de espera agotado", None, None
    except GeocoderInsufficientPrivileges:
        return "Bloqueado: Verificar API o User-Agent", None, None

# Limpieza inicial del archivo
input_file = "H:/Nueva carpeta/overpass-turbo.eu/Parques de diverciones Centros comerciales 2015.csv"
cleaned_file = "H:/Nueva carpeta/overpass-turbo.eu/Parques de diverciones Centros comerciales 20161.csv"

with open(input_file, 'r', encoding='utf-8') as infile, open(cleaned_file, 'w', encoding='utf-8') as outfile:
    for line in infile:
        if line.strip():  # Omitir líneas vacías
            outfile.write(line)

# Cargamos el archivo limpio
try:
    data = pd.read_csv(cleaned_file, sep='\t', encoding='utf-8')
except pd.errors.ParserError as e:
    print(f"Error al cargar el archivo: {e}")
    data = pd.read_csv(cleaned_file, sep=',', encoding='utf-8', on_bad_lines='skip')

# Añadimos  la columna 'name' si no existe
if 'name' not in data.columns:
    data['name'] = None

# Actualizamos las columnas addr:street, addr:city y name
def update_address(row):
    if pd.notna(row['@lat']) and pd.notna(row['@lon']):
        address, city, name = reverse_geocode(row['@lat'], row['@lon'])
        row['addr:street'] = address
        row['addr:city'] = city
        row['name'] = name
    return row

# Aplicamos la función fila por fila con pausas para respetar los límites de solicitudes
for index, row in data.iterrows():
    data.loc[index] = update_address(row)
    time.sleep(2)

# Guardamos los resultados
output_file = "H:/Nueva carpeta/overpass-turbo.eu/Parques_direcciones_actualizadas_2015.csv"
data.to_csv(output_file, index=False)
print(data.head())



                       name  \
0           Titusville Mall   
1  Landmark Shopping Center   
2            LandMark Plaza   
3                      None   
4                  CocoWalk   

                                         addr:street   addr:city addr:state  \
0  Titusville Mall, Narvaez Drive, Titusville, Br...  Titusville        NaN   
1  Landmark Shopping Center, North 20th Street, H...        None        NaN   
2  LandMark Plaza, East Fletcher Avenue, Hillsbor...        None        NaN   
3      Manatee County, Florida, 34222, United States        None        NaN   
4  CocoWalk, 3015, Grand Avenue, Dinner Key, Miam...       Miami        NaN   

   amenity       @lat       @lon  
0      NaN  28.575472 -80.802640  
1      NaN  28.068730 -82.436298  
2      NaN  28.068711 -82.437068  
3      NaN  27.534335 -82.506753  
4      NaN  25.728644 -80.242038  


Eliminamos colunas innecesarias y cambiamos los nombres

In [ ]:
# Cargamos  los datos 
archivo = "H:/Nueva carpeta/overpass-turbo.eu/Parques_direcciones_actualizadas_2015.csv" 
data = pd.read_csv(archivo)

# Renombramos  las columnas
data.rename(columns={
    'addr:street': 'direccion',
    'amenity': 'tipo',
    '@lat': 'latitud',
    '@lon': 'longitud'
}, inplace=True)

# Dividimos la columna direccion para extraer los datos
direccion_split = data['direccion'].fillna('').str.split(',', expand=True)

# Creamos las nuebas colunas a usar
data['nombre'] = direccion_split[0].str.strip()

# Extraemos la direccion combinando todas las partes de la dirección hasta el estado
data['direccion'] = direccion_split.apply(
    lambda row: ', '.join(filter(None, [row[1].strip(), row[2].strip()])), axis=1
)

# Extraemos la ciudad siempre y cuando tenga la palabra County 
data['ciudad'] = direccion_split.apply(
    lambda row: next((x.strip() for x in row if 'County' in str(x)), 'No definido'), axis=1
)

# Extraemos el estado  "Florida"
data['estado'] = direccion_split.apply(lambda row: 'Florida', axis=1)

# Extraemos  y validamos los códigos postales 
data['codigo_postal'] = direccion_split.apply(
    lambda row: next((re.search(r'\b\d{5}\b', str(x)).group(0) for x in row if re.search(r'\b\d{5}\b', str(x))), 'No definido'), axis=1
)

# Saparamos el peis "United States"
data['pais'] = direccion_split.apply(lambda row: 'United States', axis=1)

# Reordenamos las columnas 
data = data[['nombre', 'direccion', 'ciudad', 'estado', 'pais', 'codigo_postal', 'longitud', 'latitud']]

# Eliminamos  duplicados
data.drop_duplicates(inplace=True)

# Guardamos el resultado
processed_file_path = 'H:/Nueva carpeta/overpass-turbo.eu/Parques_de_diversiones_Centros_comerciales_2015.csv'
data.to_csv(processed_file_path, index=False)

# Análisamos de datos vacíos y duplicados
vacios = data.isnull().sum()
duplicados = data.duplicated().sum()

print("Análisis de datos:")
print("Valores vacíos por columna:")
print(vacios)
print(f"Número de filas duplicadas: {duplicados}")
print(data.head())


Análisis de datos:
Valores vacíos por columna:
nombre           0
direccion        0
ciudad           0
estado           0
pais             0
codigo_postal    0
longitud         0
latitud          0
dtype: int64
Número de filas duplicadas: 0
                     nombre                                  direccion  \
0           Titusville Mall                  Narvaez Drive, Titusville   
1  Landmark Shopping Center     North 20th Street, Hillsborough County   
2            LandMark Plaza  East Fletcher Avenue, Hillsborough County   
3            Manatee County                             Florida, 34222   
4                  CocoWalk                         3015, Grand Avenue   

                ciudad   estado           pais codigo_postal   longitud  \
0       Brevard County  Florida  United States         32780 -80.802640   
1  Hillsborough County  Florida  United States         33613 -82.436298   
2  Hillsborough County  Florida  United States         33613 -82.437068   
3       Manat

Eliminamos la palabra County

In [90]:
# Eliminamos la palabra 'County' de las columnas 'direccion' y 'ciudad'
data['direccion'] = data['direccion'].str.replace(r'\bCounty\b', '', regex=True).str.strip()
data['ciudad'] = data['ciudad'].str.replace(r'\bCounty\b', '', regex=True).str.strip()
print(data[['direccion', 'ciudad']].head())

# Guardamos el resultado 
processed_file_path = 'H:/Nueva carpeta/overpass-turbo.eu/Parques_de_diversiones_Centros_comerciales_2015.csv'
data.to_csv(processed_file_path, index=False)

                            direccion        ciudad
0           Narvaez Drive, Titusville       Brevard
1     North 20th Street, Hillsborough  Hillsborough
2  East Fletcher Avenue, Hillsborough  Hillsborough
3                      Florida, 34222       Manatee
4                  3015, Grand Avenue    Miami-Dade


Creamos las ids para nombre y ciudad

In [ ]:
# Cargamos las rutas de los archivos
input_path = r"H:/Nueva carpeta/overpass-turbo.eu/Parques_de_diversiones_Centros_comerciales_2015.csv"
output_path = r"H:/Nueva carpeta/overpass-turbo.eu/Parques_de_diversiones_Centros_comerciales_2015_final.csv"
nombres_dim_path = "H:/Nueva carpeta/overpass-turbo.eu/nombres_dim.csv"
ciudades_dim_path = "H:/Nueva carpeta/overpass-turbo.eu/ciudades_dim.csv"

# Cargamos los catálogos
def load_catalog(path, key, value):
    catalog = {}
    if os.path.exists(path):
        with open(path, mode='r', encoding='utf-8') as f:
            reader = csv.DictReader(f)
            for row in reader:
                catalog[row[key]] = int(row[value])
    return catalog

# Guardamos los catálogos de ciudad y nombres
def save_catalog(path, catalog, key, value):
    with open(path, mode='w', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=[value, key])
        writer.writeheader()
        for k, v in catalog.items():
            writer.writerow({value: v, key: k})

# Generamos IDs únicos
def get_id(value, catalog):
    if value not in catalog:
        catalog[value] = len(catalog) + 1
    return catalog[value]

# Cargamos catálogos existentes
nombres_dict = load_catalog(nombres_dim_path, "nombre", "id_nombre")
ciudades_dict = load_catalog(ciudades_dim_path, "ciudad", "id_ciudad")

# Procesamos archivo de entrada
def process_file(input_path, output_path):
    if not os.path.exists(input_path):
        raise FileNotFoundError(f"El archivo de entrada no existe: {input_path}")

    with open(input_path, mode='r', encoding='utf-8') as infile:
        reader = csv.DictReader(infile)
        original_fieldnames = reader.fieldnames

        # Validamos las columnas columnas requeridas
        required_cols = {"nombre", "direccion", "latitud", "longitud", "ciudad", "estado", "pais", "codigo_postal"}
        missing_cols = required_cols - set(original_fieldnames)
        if missing_cols:
            raise ValueError(f"El CSV no contiene las siguientes columnas requeridas: {', '.join(missing_cols)}")

        # Nombres de las columnas
        fieldnames = [
            "id_nombre", "nombre", "direccion", "latitud", "longitud",
            "id_ciudad", "ciudad", "estado", "pais", "codigo_postal"
        ]

        # ordenamos y creamos el archivo de salida
        with open(output_path, mode='w', newline='', encoding='utf-8') as outfile:
            writer = csv.DictWriter(outfile, fieldnames=fieldnames)
            writer.writeheader()

            for row in reader:
                id_nombre = get_id(row["nombre"], nombres_dict)
                id_ciudad = get_id(row["ciudad"], ciudades_dict)

                new_row = {
                    "id_nombre": id_nombre,
                    "nombre": row["nombre"],
                    "direccion": row["direccion"],
                    "latitud": row["latitud"],
                    "longitud": row["longitud"],
                    "id_ciudad": id_ciudad,
                    "ciudad": row["ciudad"],
                    "estado": row["estado"],
                    "pais": row["pais"],
                    "codigo_postal": row["codigo_postal"]
                }
                writer.writerow(new_row)

process_file(input_path, output_path)

# Guardamos el catálogos actualizados 
save_catalog(nombres_dim_path, nombres_dict, "nombre", "id_nombre")
save_catalog(ciudades_dim_path, ciudades_dict, "ciudad", "id_ciudad")

print("Proceso completado con éxito.")



Proceso completado con éxito.


Para el area de ML se combierte el archivo a formato .parquet

In [93]:
import pandas as pd

# Cargamos el archivo CSV 
csv_path = r"H:\Nueva carpeta\overpass-turbo.eu\Parques_de_diversiones_Centros_comerciales_2015_final.csv"

# Ruta del archivo Parquet de salida
parquet_path = r"H:\Nueva carpeta\overpass-turbo.eu\Parques_de_diversiones_Centros_comerciales_2015_final.parquet"

df = pd.read_csv(csv_path)

# Guardamos el archivo Parquet
df.to_parquet(parquet_path, engine='pyarrow', index=False)

print(f"Archivo convertido exitosamente a Parquet: {parquet_path}")

Archivo convertido exitosamente a Parquet: H:\Nueva carpeta\overpass-turbo.eu\Parques_de_diversiones_Centros_comerciales_2015_final.parquet


Parques de diverciones Centros comerciales 2016

In [56]:
# 1. Cargamos los datos
file_path = r'H:/Nueva carpeta/overpass-turbo.eu/Parques de diverciones Centros comerciales 2016.csv'  
data = pd.read_csv(file_path, delimiter='\t')  

# 2. Realizamos una revisión general
print("Dimensiones del dataset:", data.shape)
print("\nNombres de las columnas:", data.columns.tolist())
print("\nPrimeras filas del dataset:")
print(data.head())

# 3. Mostramos la calidad de los datos
print("\nValores nulos por columna:")
print(data.isnull().sum())

print("\nTipos de datos:")
print(data.dtypes)

# 4. Realisaos una estadísticas descriptivas
print("\nEstadísticas descriptivas para columnas numéricas:")
print(data.describe())

print("\nValores únicos por columna:")
print(data.nunique())

Dimensiones del dataset: (348, 7)

Nombres de las columnas: ['name', 'addr:street', 'addr:city', 'addr:state', 'amenity', '@lat', '@lon']

Primeras filas del dataset:
                       name              addr:street      addr:city  \
0           Titusville Mall                      NaN            NaN   
1  Landmark Shopping Center                      NaN            NaN   
2            LandMark Plaza                      NaN            NaN   
3  Ellenton Premium Outlets  Factory Shops Boulevard       Ellenton   
4                  CocoWalk             Grand Avenue  Coconut Grove   

  addr:state  amenity       @lat       @lon  
0        NaN      NaN  28.575472 -80.802640  
1        NaN      NaN  28.068730 -82.436298  
2        NaN      NaN  28.068711 -82.437068  
3        NaN      NaN  27.534335 -82.506753  
4        NaN      NaN  25.728644 -80.242038  

Valores nulos por columna:
name           156
addr:street    247
addr:city      254
addr:state     269
amenity        348
@lat   

Descargamos las direcciones en base ala latitud y longitud

In [ ]:
# Configuramos el geolocalizador con un user_agent único
geolocator = Nominatim(user_agent="mi_proyecto_geocodificacion_v1")

# Función para realizar la geocodificación inversa
def reverse_geocode(lat, lon):
    try:
        location = geolocator.reverse((lat, lon), timeout=10)
        if location:
            # Extraemos  dirección completa y ciudad
            address = location.address
            city = location.raw.get('address', {}).get('city', None)
            return address, city
        else:
            return "Dirección no encontrada", None
    except GeocoderTimedOut:
        return "Tiempo de espera agotado", None
    except GeocoderInsufficientPrivileges:
        return "Bloqueado: Verificar API o User-Agent", None

# Actualizamos las columnas addr:street y addr:city
def update_address(row):
    if pd.notna(row['@lat']) and pd.notna(row['@lon']):
        address, city = reverse_geocode(row['@lat'], row['@lon'])
        row['addr:street'] = address  # Guardar dirección completa
        row['addr:city'] = city       # Actualizar ciudad
    return row

# Aplicamos una pausa de 2 segundos fila por fila con retraso para respetar los límites de solisitudes
for index, row in data.iterrows():
    data.loc[index] = update_address(row)
    time.sleep(2)

# Guardamos los resultados en un nuevo archivo CSV
data.to_csv("H:/Nueva carpeta/overpass-turbo.eu/Parques de diverciones Centros comerciales 2016.csv", index=False)
print(data[['@lat', '@lon', 'addr:street', 'addr:city']].head())


        @lat       @lon                                        addr:street  \
0  28.575472 -80.802640  Titusville Mall, Narvaez Drive, Titusville, Br...   
1  28.068730 -82.436298  Landmark Shopping Center, North 20th Street, H...   
2  28.068711 -82.437068  LandMark Plaza, East Fletcher Avenue, Hillsbor...   
3  27.534335 -82.506753      Manatee County, Florida, 34222, United States   
4  25.728644 -80.242038  CocoWalk, 3015, Grand Avenue, Dinner Key, Miam...   

    addr:city  
0  Titusville  
1        None  
2        None  
3        None  
4       Miami  


Eliminamos colunas innecesarias y cambiamos los nombres

In [58]:
# Cambiamos nombres de columnas para mejor comprencion
data.rename(columns={
    'name': 'nombre',
    'addr:street': 'direccion',
    '@lat': 'latitud',
    '@lon': 'longitud'
}, inplace=True)

# Creamos nuevas columnas para ciudad, estado, país y código postal
data['ciudad'] = data['direccion'].str.extract(r', ([^,]+), [^,]+ County')[0]
data['estado'] = data['direccion'].str.extract(r', ([^,]+), \d{5}, United States')[0]
data['pais'] = 'United States'  # Dado que todos los datos son de EE.UU.
data['codigo_postal'] = data['direccion'].str.extract(r', (\d{5}), United States')[0]

# Limpiamos la columna 'direccion' para que quede solo la dirección
data['direccion'] = data['direccion'].str.extract(r'^([^,]+)')[0]

# Convertimos todas las columnas a texto para garantizar compatibilidad
data = data.astype(str)

# Rellenamos valores faltantes con 'None'
data.fillna('None', inplace=True)

# Eliminamos columnas no deseadas
data.drop(columns=['addr:city', 'addr:state', 'amenity'], inplace=True, errors='ignore')

# Eliminamos duplicados
data.drop_duplicates(inplace=True)

# Guardamos el archivo procesado
processed_file_path = 'H:/Nueva carpeta/overpass-turbo.eu/Parques de diverciones Centros comerciales 2016.csv'
data.to_csv(processed_file_path, index=False)

# Realizamos un análisis de datos vacíos y duplicados
vacios = data.isnull().sum()
duplicados = data.duplicated().sum()

print("Análisis de datos:")
print("Valores vacíos por columna:")
print(vacios)
print(f"Número de filas duplicadas: {duplicados}")
print(data.head())


Análisis de datos:
Valores vacíos por columna:
nombre           0
direccion        0
latitud          0
longitud         0
ciudad           0
estado           0
pais             0
codigo_postal    0
dtype: int64
Número de filas duplicadas: 0
                     nombre                 direccion     latitud  \
0           Titusville Mall           Titusville Mall  28.5754724   
1  Landmark Shopping Center  Landmark Shopping Center  28.0687299   
2            LandMark Plaza            LandMark Plaza   28.068711   
3  Ellenton Premium Outlets            Manatee County  27.5343346   
4                  CocoWalk                  CocoWalk  25.7286438   

      longitud                ciudad   estado           pais codigo_postal  
0    -80.80264            Titusville  Florida  United States         32780  
1   -82.436298     North 20th Street  Florida  United States         33613  
2  -82.4370678  East Fletcher Avenue  Florida  United States         33613  
3   -82.506753                   na

Creamos las ids para nombre y ciudad

In [60]:
# Rutas de los archivos
input_path = r"H:/Nueva carpeta/overpass-turbo.eu/Parques de diverciones Centros comerciales 2016.csv"
output_path = r"H:/Nueva carpeta/overpass-turbo.eu/Parques_de_diversiones_Centros_comerciales_2016_final.csv"
nombres_dim_path = "H:/Nueva carpeta/overpass-turbo.eu/nombres_dim.csv"
ciudades_dim_path = "H:/Nueva carpeta/overpass-turbo.eu/ciudades_dim.csv"

# Cargar catálogos
def load_catalog(path, key, value):
    catalog = {}
    if os.path.exists(path):
        with open(path, mode='r', encoding='utf-8') as f:
            reader = csv.DictReader(f)
            for row in reader:
                catalog[row[key]] = int(row[value])
    return catalog

# Guardar catálogos
def save_catalog(path, catalog, key, value):
    with open(path, mode='w', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=[value, key])
        writer.writeheader()
        for k, v in catalog.items():
            writer.writerow({value: v, key: k})

# Generar IDs únicos
def get_id(value, catalog):
    if value not in catalog:
        catalog[value] = len(catalog) + 1
    return catalog[value]

# Cargar catálogos existentes
nombres_dict = load_catalog(nombres_dim_path, "nombre", "id_nombre")
ciudades_dict = load_catalog(ciudades_dim_path, "ciudad", "id_ciudad")

# Procesar archivo de entrada
def process_file(input_path, output_path):
    if not os.path.exists(input_path):
        raise FileNotFoundError(f"El archivo de entrada no existe: {input_path}")

    with open(input_path, mode='r', encoding='utf-8') as infile:
        reader = csv.DictReader(infile)
        original_fieldnames = reader.fieldnames

        # Validar columnas requeridas
        required_cols = {"nombre", "direccion", "latitud", "longitud", "ciudad", "estado", "pais", "codigo_postal"}
        missing_cols = required_cols - set(original_fieldnames)
        if missing_cols:
            raise ValueError(f"El CSV no contiene las siguientes columnas requeridas: {', '.join(missing_cols)}")

        # Columnas de salida
        fieldnames = [
            "id_nombre", "nombre", "direccion", "latitud", "longitud",
            "id_ciudad", "ciudad", "estado", "pais", "codigo_postal"
        ]

        # Escribir archivo de salida
        with open(output_path, mode='w', newline='', encoding='utf-8') as outfile:
            writer = csv.DictWriter(outfile, fieldnames=fieldnames)
            writer.writeheader()

            for row in reader:
                id_nombre = get_id(row["nombre"], nombres_dict)
                id_ciudad = get_id(row["ciudad"], ciudades_dict)

                new_row = {
                    "id_nombre": id_nombre,
                    "nombre": row["nombre"],
                    "direccion": row["direccion"],
                    "latitud": row["latitud"],
                    "longitud": row["longitud"],
                    "id_ciudad": id_ciudad,
                    "ciudad": row["ciudad"],
                    "estado": row["estado"],
                    "pais": row["pais"],
                    "codigo_postal": row["codigo_postal"]
                }
                writer.writerow(new_row)

# Ejecutar procesamiento
process_file(input_path, output_path)

# Guardar catálogos actualizados
save_catalog(nombres_dim_path, nombres_dict, "nombre", "id_nombre")
save_catalog(ciudades_dim_path, ciudades_dict, "ciudad", "id_ciudad")

print("Proceso completado con éxito.")

Proceso completado con éxito.


Parques de diverciones Centros comerciales 2017

In [58]:
# 1. Cargamos los datos
file_path = r'H:/Nueva carpeta/overpass-turbo.eu/Parques de diverciones Centros comerciales 2017.csv'  
data = pd.read_csv(file_path, delimiter='\t')  

# 2. Realizamos una revisión general
print("Dimensiones del dataset:", data.shape)
print("\nNombres de las columnas:", data.columns.tolist())
print("\nPrimeras filas del dataset:")
print(data.head())

# 3. Mostramos la calidad de los datos
print("\nValores nulos por columna:")
print(data.isnull().sum())

print("\nTipos de datos:")
print(data.dtypes)

# 4. Realisaos una estadísticas descriptivas
print("\nEstadísticas descriptivas para columnas numéricas:")
print(data.describe())

print("\nValores únicos por columna:")
print(data.nunique())


Dimensiones del dataset: (337, 7)

Nombres de las columnas: ['name', 'addr:street', 'addr:city', 'addr:state', 'amenity', '@lat', '@lon']

Primeras filas del dataset:
                       name              addr:street      addr:city  \
0           Titusville Mall                      NaN            NaN   
1  Landmark Shopping Center                      NaN            NaN   
2            LandMark Plaza                      NaN            NaN   
3  Ellenton Premium Outlets  Factory Shops Boulevard       Ellenton   
4                  CocoWalk             Grand Avenue  Coconut Grove   

  addr:state  amenity       @lat       @lon  
0        NaN      NaN  28.575472 -80.802640  
1        NaN      NaN  28.068730 -82.436298  
2        NaN      NaN  28.068711 -82.437068  
3        NaN      NaN  27.534335 -82.506753  
4        NaN      NaN  25.728644 -80.242038  

Valores nulos por columna:
name           148
addr:street    238
addr:city      244
addr:state     259
amenity        337
@lat   

In [61]:

# Configuramos el geolocalizador con un user_agent único
geolocator = Nominatim(user_agent="mi_proyecto_geocodificacion_v1")

# Función para realizar la geocodificación inversa y buscar el nombre del lugar
def reverse_geocode(lat, lon):
    try:
        location = geolocator.reverse((lat, lon), timeout=10)
        if location:
            address = location.address
            city = location.raw.get('address', {}).get('city', None)
            name = location.raw.get('name', None) or location.raw.get('address', {}).get('attraction', None)
            return address, city, name
        else:
            return "Dirección no encontrada", None, None
    except GeocoderTimedOut:
        return "Tiempo de espera agotado", None, None
    except GeocoderInsufficientPrivileges:
        return "Bloqueado: Verificar API o User-Agent", None, None

# Limpieza inicial del archivo
input_file = "H:/Nueva carpeta/overpass-turbo.eu/Parques de diverciones Centros comerciales 2017.csv"
cleaned_file = "H:/Nueva carpeta/overpass-turbo.eu/Parques de diverciones Centros comerciales 20171.csv"

with open(input_file, 'r', encoding='utf-8') as infile, open(cleaned_file, 'w', encoding='utf-8') as outfile:
    for line in infile:
        if line.strip():  # Omitir líneas vacías
            outfile.write(line)

# Cargar el archivo limpio
try:
    data = pd.read_csv(cleaned_file, sep='\t', encoding='utf-8')
except pd.errors.ParserError as e:
    print(f"Error al cargar el archivo: {e}")
    data = pd.read_csv(cleaned_file, sep=',', encoding='utf-8', on_bad_lines='skip')

# Añadir la columna 'name' si no existe
if 'name' not in data.columns:
    data['name'] = None

# Actualizar las columnas addr:street, addr:city y name
def update_address(row):
    if pd.notna(row['@lat']) and pd.notna(row['@lon']):
        address, city, name = reverse_geocode(row['@lat'], row['@lon'])
        row['addr:street'] = address
        row['addr:city'] = city
        row['name'] = name
    return row

# Aplicar la función fila por fila con pausas para respetar límites de solicitudes
for index, row in data.iterrows():
    data.loc[index] = update_address(row)
    time.sleep(2)

# Guardar los resultados
output_file = "H:/Nueva carpeta/overpass-turbo.eu/Parques_direcciones_actualizadas_2017.csv"
data.to_csv(output_file, index=False)

# Mostrar las primeras filas procesadas
print(data.head())

                       name  \
0           Titusville Mall   
1  Landmark Shopping Center   
2            LandMark Plaza   
3                      None   
4                  CocoWalk   

                                         addr:street   addr:city addr:state  \
0  Titusville Mall, Narvaez Drive, Titusville, Br...  Titusville        NaN   
1  Landmark Shopping Center, North 20th Street, H...        None        NaN   
2  LandMark Plaza, East Fletcher Avenue, Hillsbor...        None        NaN   
3      Manatee County, Florida, 34222, United States        None        NaN   
4  CocoWalk, 3015, Grand Avenue, Dinner Key, Miam...       Miami        NaN   

   amenity       @lat       @lon  
0      NaN  28.575472 -80.802640  
1      NaN  28.068730 -82.436298  
2      NaN  28.068711 -82.437068  
3      NaN  27.534335 -82.506753  
4      NaN  25.728644 -80.242038  


Eliminamos columnas innesesarias 

In [ ]:
# Cargamos  los datos 
archivo = "H:/Nueva carpeta/overpass-turbo.eu/Parques_direcciones_actualizadas_2017.csv" 
data = pd.read_csv(archivo)

# Renombramos  las columnas
data.rename(columns={
    'addr:street': 'direccion',
    'amenity': 'tipo',
    '@lat': 'latitud',
    '@lon': 'longitud'
}, inplace=True)

# Dividimos la columna direccion para extraer los datos
direccion_split = data['direccion'].fillna('').str.split(',', expand=True)

# Creamos las nuebas colunas a usar
data['nombre'] = direccion_split[0].str.strip()

# Extraemos la direccion combinando todas las partes de la dirección hasta el estado
data['direccion'] = direccion_split.apply(
    lambda row: ', '.join(filter(None, [row[1].strip(), row[2].strip()])), axis=1
)

# Extraemos la ciudad siempre y cuando tenga la palabra County 
data['ciudad'] = direccion_split.apply(
    lambda row: next((x.strip() for x in row if 'County' in str(x)), 'No definido'), axis=1
)

# Extraemos el estado  "Florida"
data['estado'] = direccion_split.apply(lambda row: 'Florida', axis=1)

# Extraemos  y validamos los códigos postales 
data['codigo_postal'] = direccion_split.apply(
    lambda row: next((re.search(r'\b\d{5}\b', str(x)).group(0) for x in row if re.search(r'\b\d{5}\b', str(x))), 'No definido'), axis=1
)

# Saparamos el peis "United States"
data['pais'] = direccion_split.apply(lambda row: 'United States', axis=1)

# Reordenamos las columnas 
data = data[['nombre', 'direccion', 'ciudad', 'estado', 'pais', 'codigo_postal', 'longitud', 'latitud']]

# Eliminamos  duplicados
data.drop_duplicates(inplace=True)

# Guardamos el resultado
processed_file_path = 'H:/Nueva carpeta/overpass-turbo.eu/Parques_de_diversiones_Centros_comerciales_2017.csv'
data.to_csv(processed_file_path, index=False)

# Análisamos de datos vacíos y duplicados
vacios = data.isnull().sum()
duplicados = data.duplicated().sum()

print("Análisis de datos:")
print("Valores vacíos por columna:")
print(vacios)
print(f"Número de filas duplicadas: {duplicados}")
print(data.head())


Análisis de datos:
Valores vacíos por columna:
nombre           0
direccion        0
ciudad           0
estado           0
pais             0
codigo_postal    0
longitud         0
latitud          0
dtype: int64
Número de filas duplicadas: 0
                     nombre                                  direccion  \
0           Titusville Mall                  Narvaez Drive, Titusville   
1  Landmark Shopping Center     North 20th Street, Hillsborough County   
2            LandMark Plaza  East Fletcher Avenue, Hillsborough County   
3            Manatee County                             Florida, 34222   
4                  CocoWalk                         3015, Grand Avenue   

                ciudad   estado           pais codigo_postal   longitud  \
0       Brevard County  Florida  United States         32780 -80.802640   
1  Hillsborough County  Florida  United States         33613 -82.436298   
2  Hillsborough County  Florida  United States         33613 -82.437068   
3       Manat

In [ ]:
# Eliminamos la palabra 'County' de las columnas 'direccion' y 'ciudad'
data['direccion'] = data['direccion'].str.replace(r'\bCounty\b', '', regex=True).str.strip()
data['ciudad'] = data['ciudad'].str.replace(r'\bCounty\b', '', regex=True).str.strip()
print(data[['direccion', 'ciudad']].head())

# Guardamos el resultado 
processed_file_path = 'H:/Nueva carpeta/overpass-turbo.eu/Parques_de_diversiones_Centros_comerciales_2017.csv'
data.to_csv(processed_file_path, index=False)


                            direccion        ciudad
0           Narvaez Drive, Titusville       Brevard
1     North 20th Street, Hillsborough  Hillsborough
2  East Fletcher Avenue, Hillsborough  Hillsborough
3                      Florida, 34222       Manatee
4                  3015, Grand Avenue    Miami-Dade


Creamos Id nombre e id Ciudad

In [ ]:
# Cargamos las rutas de los archivos
input_path = r"H:/Nueva carpeta/overpass-turbo.eu/Parques_de_diversiones_Centros_comerciales_2017.csv"
output_path = r"H:/Nueva carpeta/overpass-turbo.eu/Parques_de_diversiones_Centros_comerciales_2017_final.csv"
nombres_dim_path = "H:/Nueva carpeta/overpass-turbo.eu/nombres_dim.csv"
ciudades_dim_path = "H:/Nueva carpeta/overpass-turbo.eu/ciudades_dim.csv"

# Cargamos los catálogos
def load_catalog(path, key, value):
    catalog = {}
    if os.path.exists(path):
        with open(path, mode='r', encoding='utf-8') as f:
            reader = csv.DictReader(f)
            for row in reader:
                catalog[row[key]] = int(row[value])
    return catalog

# Guardamos los catálogos de ciudad y nombres
def save_catalog(path, catalog, key, value):
    with open(path, mode='w', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=[value, key])
        writer.writeheader()
        for k, v in catalog.items():
            writer.writerow({value: v, key: k})

# Generamos IDs únicos
def get_id(value, catalog):
    if value not in catalog:
        catalog[value] = len(catalog) + 1
    return catalog[value]

# Cargamos catálogos existentes
nombres_dict = load_catalog(nombres_dim_path, "nombre", "id_nombre")
ciudades_dict = load_catalog(ciudades_dim_path, "ciudad", "id_ciudad")

# Procesamos archivo de entrada
def process_file(input_path, output_path):
    if not os.path.exists(input_path):
        raise FileNotFoundError(f"El archivo de entrada no existe: {input_path}")

    with open(input_path, mode='r', encoding='utf-8') as infile:
        reader = csv.DictReader(infile)
        original_fieldnames = reader.fieldnames

        # Validamos las columnas columnas requeridas
        required_cols = {"nombre", "direccion", "latitud", "longitud", "ciudad", "estado", "pais", "codigo_postal"}
        missing_cols = required_cols - set(original_fieldnames)
        if missing_cols:
            raise ValueError(f"El CSV no contiene las siguientes columnas requeridas: {', '.join(missing_cols)}")

        # Nombres de las columnas
        fieldnames = [
            "id_nombre", "nombre", "direccion", "latitud", "longitud",
            "id_ciudad", "ciudad", "estado", "pais", "codigo_postal"
        ]

        # ordenamos y creamos el archivo de salida
        with open(output_path, mode='w', newline='', encoding='utf-8') as outfile:
            writer = csv.DictWriter(outfile, fieldnames=fieldnames)
            writer.writeheader()

            for row in reader:
                id_nombre = get_id(row["nombre"], nombres_dict)
                id_ciudad = get_id(row["ciudad"], ciudades_dict)

                new_row = {
                    "id_nombre": id_nombre,
                    "nombre": row["nombre"],
                    "direccion": row["direccion"],
                    "latitud": row["latitud"],
                    "longitud": row["longitud"],
                    "id_ciudad": id_ciudad,
                    "ciudad": row["ciudad"],
                    "estado": row["estado"],
                    "pais": row["pais"],
                    "codigo_postal": row["codigo_postal"]
                }
                writer.writerow(new_row)

process_file(input_path, output_path)

# Guardamos el catálogos actualizados 
save_catalog(nombres_dim_path, nombres_dict, "nombre", "id_nombre")
save_catalog(ciudades_dim_path, ciudades_dict, "ciudad", "id_ciudad")

print("Proceso completado con éxito.")

Proceso completado con éxito.


Para el area de ML se combierte el archivo a formato .parquet

In [76]:
# Cargamos el archivo CSV 
csv_path = r"H:/Nueva carpeta/overpass-turbo.eu/Parques_de_diversiones_Centros_comerciales_2017_final.csv"

# Ruta del archivo Parquet de salida
parquet_path = r"H:\Nueva carpeta\overpass-turbo.eu\Parques_de_diversiones_Centros_comerciales_2017_final.parquet"

df = pd.read_csv(csv_path)

# Guardamos el archivo Parquet
df.to_parquet(parquet_path, engine='pyarrow', index=False)

print(f"Archivo convertido exitosamente a Parquet: {parquet_path}")

Archivo convertido exitosamente a Parquet: H:\Nueva carpeta\overpass-turbo.eu\Parques_de_diversiones_Centros_comerciales_2017_final.parquet


Parques de diverciones Centros comerciales 2016

In [62]:
# 1. Cargamos los datos
file_path = r'H:/Nueva carpeta/overpass-turbo.eu/Parques de diverciones Centros comerciales 2016.csv'  
data = pd.read_csv(file_path, delimiter='\t')  

# 2. Realizamos una revisión general
print("Dimensiones del dataset:", data.shape)
print("\nNombres de las columnas:", data.columns.tolist())
print("\nPrimeras filas del dataset:")
print(data.head())

# 3. Mostramos la calidad de los datos
print("\nValores nulos por columna:")
print(data.isnull().sum())

print("\nTipos de datos:")
print(data.dtypes)

# 4. Realisaos una estadísticas descriptivas
print("\nEstadísticas descriptivas para columnas numéricas:")
print(data.describe())

print("\nValores únicos por columna:")
print(data.nunique())

Dimensiones del dataset: (348, 7)

Nombres de las columnas: ['name', 'addr:street', 'addr:city', 'addr:state', 'amenity', '@lat', '@lon']

Primeras filas del dataset:
                       name              addr:street      addr:city  \
0           Titusville Mall                      NaN            NaN   
1  Landmark Shopping Center                      NaN            NaN   
2            LandMark Plaza                      NaN            NaN   
3  Ellenton Premium Outlets  Factory Shops Boulevard       Ellenton   
4                  CocoWalk             Grand Avenue  Coconut Grove   

  addr:state  amenity       @lat       @lon  
0        NaN      NaN  28.575472 -80.802640  
1        NaN      NaN  28.068730 -82.436298  
2        NaN      NaN  28.068711 -82.437068  
3        NaN      NaN  27.534335 -82.506753  
4        NaN      NaN  25.728644 -80.242038  

Valores nulos por columna:
name           156
addr:street    247
addr:city      254
addr:state     269
amenity        348
@lat   

Descargamos direciones en base a la latitud y longitud

In [ ]:
# Configuramos el geolocalizador con un user_agent único
geolocator = Nominatim(user_agent="mi_proyecto_geocodificacion_v1")

# Función para realizar la geocodificación inversa y buscar el nombre del lugar
def reverse_geocode(lat, lon):
    try:
        location = geolocator.reverse((lat, lon), timeout=10)
        if location:
            address = location.address
            city = location.raw.get('address', {}).get('city', None)
            name = location.raw.get('name', None) or location.raw.get('address', {}).get('attraction', None)
            return address, city, name
        else:
            return "Dirección no encontrada", None, None
    except GeocoderTimedOut:
        return "Tiempo de espera agotado", None, None
    except GeocoderInsufficientPrivileges:
        return "Bloqueado: Verificar API o User-Agent", None, None

# Limpieza inicial del archivo
input_file = "H:/Nueva carpeta/overpass-turbo.eu/Parques de diverciones Centros comerciales 2016.csv"
cleaned_file = "H:/Nueva carpeta/overpass-turbo.eu/Parques de diverciones Centros comerciales 20161.csv"

with open(input_file, 'r', encoding='utf-8') as infile, open(cleaned_file, 'w', encoding='utf-8') as outfile:
    for line in infile:
        if line.strip():  # Omitir líneas vacías
            outfile.write(line)

# Cargamos el archivo limpio
try:
    data = pd.read_csv(cleaned_file, sep='\t', encoding='utf-8')
except pd.errors.ParserError as e:
    print(f"Error al cargar el archivo: {e}")
    data = pd.read_csv(cleaned_file, sep=',', encoding='utf-8', on_bad_lines='skip')

# Añadimos  la columna 'name' si no existe
if 'name' not in data.columns:
    data['name'] = None

# Actualizamos las columnas addr:street, addr:city y name
def update_address(row):
    if pd.notna(row['@lat']) and pd.notna(row['@lon']):
        address, city, name = reverse_geocode(row['@lat'], row['@lon'])
        row['addr:street'] = address
        row['addr:city'] = city
        row['name'] = name
    return row

# Aplicar la función fila por fila con pausas para respetar los límites de solicitudes
for index, row in data.iterrows():
    data.loc[index] = update_address(row)
    time.sleep(2)

# Guardar los resultados
output_file = "H:/Nueva carpeta/overpass-turbo.eu/Parques_direcciones_actualizadas_2016.csv"
data.to_csv(output_file, index=False)

# Mostrar las primeras filas procesadas
print(data.head())

                       name  \
0           Titusville Mall   
1  Landmark Shopping Center   
2            LandMark Plaza   
3                      None   
4                  CocoWalk   

                                         addr:street   addr:city addr:state  \
0  Titusville Mall, Narvaez Drive, Titusville, Br...  Titusville        NaN   
1  Landmark Shopping Center, North 20th Street, H...        None        NaN   
2  LandMark Plaza, East Fletcher Avenue, Hillsbor...        None        NaN   
3      Manatee County, Florida, 34222, United States        None        NaN   
4  CocoWalk, 3015, Grand Avenue, Dinner Key, Miam...       Miami        NaN   

   amenity       @lat       @lon  
0      NaN  28.575472 -80.802640  
1      NaN  28.068730 -82.436298  
2      NaN  28.068711 -82.437068  
3      NaN  27.534335 -82.506753  
4      NaN  25.728644 -80.242038  


Extremos los datos descargados en la columna direcciones , eliminamos columnas inesesarias 

In [ ]:
# Cargamos  los datos 
archivo = "H:/Nueva carpeta/overpass-turbo.eu/Parques_direcciones_actualizadas_2016.csv" 
data = pd.read_csv(archivo)

# Renombramos  las columnas
data.rename(columns={
    'addr:street': 'direccion',
    'amenity': 'tipo',
    '@lat': 'latitud',
    '@lon': 'longitud'
}, inplace=True)

# Dividimos la columna direccion para extraer los datos
direccion_split = data['direccion'].fillna('').str.split(',', expand=True)

# Creamos las nuebas colunas a usar
data['nombre'] = direccion_split[0].str.strip()

# Extraemos la direccion combinando todas las partes de la dirección hasta el estado
data['direccion'] = direccion_split.apply(
    lambda row: ', '.join(filter(None, [row[1].strip(), row[2].strip()])), axis=1
)

# Extraemos la ciudad siempre y cuando tenga la palabra County 
data['ciudad'] = direccion_split.apply(
    lambda row: next((x.strip() for x in row if 'County' in str(x)), 'No definido'), axis=1
)

# Extraemos el estado  "Florida"
data['estado'] = direccion_split.apply(lambda row: 'Florida', axis=1)

# Extraemos  y validamos los códigos postales 
data['codigo_postal'] = direccion_split.apply(
    lambda row: next((re.search(r'\b\d{5}\b', str(x)).group(0) for x in row if re.search(r'\b\d{5}\b', str(x))), 'No definido'), axis=1
)

# Saparamos el peis "United States"
data['pais'] = direccion_split.apply(lambda row: 'United States', axis=1)

# Reordenamos las columnas 
data = data[['nombre', 'direccion', 'ciudad', 'estado', 'pais', 'codigo_postal', 'longitud', 'latitud']]

# Eliminamos  duplicados
data.drop_duplicates(inplace=True)

# Guardamos el resultado
processed_file_path = 'H:/Nueva carpeta/overpass-turbo.eu/Parques_de_diversiones_Centros_comerciales_2016.csv'
data.to_csv(processed_file_path, index=False)

# Análisamos de datos vacíos y duplicados
vacios = data.isnull().sum()
duplicados = data.duplicated().sum()

print("Análisis de datos:")
print("Valores vacíos por columna:")
print(vacios)
print(f"Número de filas duplicadas: {duplicados}")
print(data.head())

Análisis de datos:
Valores vacíos por columna:
nombre           0
direccion        0
ciudad           0
estado           0
pais             0
codigo_postal    0
longitud         0
latitud          0
dtype: int64
Número de filas duplicadas: 0
                     nombre                                  direccion  \
0           Titusville Mall                  Narvaez Drive, Titusville   
1  Landmark Shopping Center     North 20th Street, Hillsborough County   
2            LandMark Plaza  East Fletcher Avenue, Hillsborough County   
3            Manatee County                             Florida, 34222   
4                  CocoWalk                         3015, Grand Avenue   

                ciudad   estado           pais codigo_postal   longitud  \
0       Brevard County  Florida  United States         32780 -80.802640   
1  Hillsborough County  Florida  United States         33613 -82.436298   
2  Hillsborough County  Florida  United States         33613 -82.437068   
3       Manat

Eliminamos la palabra County 

In [65]:
4
# Eliminamos la palabra 'County' de las columnas 'direccion' y 'ciudad'
data['direccion'] = data['direccion'].str.replace(r'\bCounty\b', '', regex=True).str.strip()
data['ciudad'] = data['ciudad'].str.replace(r'\bCounty\b', '', regex=True).str.strip()
print(data[['direccion', 'ciudad']].head())

# Guardamos el resultado 
processed_file_path = 'H:/Nueva carpeta/overpass-turbo.eu/Parques_de_diversiones_Centros_comerciales_2016.csv'
data.to_csv(processed_file_path, index=False)

                            direccion        ciudad
0           Narvaez Drive, Titusville       Brevard
1     North 20th Street, Hillsborough  Hillsborough
2  East Fletcher Avenue, Hillsborough  Hillsborough
3                      Florida, 34222       Manatee
4                  3015, Grand Avenue    Miami-Dade


Creamos id_nombre y Id_ciudad

In [66]:
# Cargamos las rutas de los archivos
input_path = r"H:/Nueva carpeta/overpass-turbo.eu/Parques_de_diversiones_Centros_comerciales_2016.csv"
output_path = r"H:/Nueva carpeta/overpass-turbo.eu/Parques_de_diversiones_Centros_comerciales_2016_final.csv"
nombres_dim_path = "H:/Nueva carpeta/overpass-turbo.eu/nombres_dim.csv"
ciudades_dim_path = "H:/Nueva carpeta/overpass-turbo.eu/ciudades_dim.csv"

# Cargamos los catálogos
def load_catalog(path, key, value):
    catalog = {}
    if os.path.exists(path):
        with open(path, mode='r', encoding='utf-8') as f:
            reader = csv.DictReader(f)
            for row in reader:
                catalog[row[key]] = int(row[value])
    return catalog

# Guardamos los catálogos de ciudad y nombres
def save_catalog(path, catalog, key, value):
    with open(path, mode='w', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=[value, key])
        writer.writeheader()
        for k, v in catalog.items():
            writer.writerow({value: v, key: k})

# Generamos IDs únicos
def get_id(value, catalog):
    if value not in catalog:
        catalog[value] = len(catalog) + 1
    return catalog[value]

# Cargamos catálogos existentes
nombres_dict = load_catalog(nombres_dim_path, "nombre", "id_nombre")
ciudades_dict = load_catalog(ciudades_dim_path, "ciudad", "id_ciudad")

# Procesamos archivo de entrada
def process_file(input_path, output_path):
    if not os.path.exists(input_path):
        raise FileNotFoundError(f"El archivo de entrada no existe: {input_path}")

    with open(input_path, mode='r', encoding='utf-8') as infile:
        reader = csv.DictReader(infile)
        original_fieldnames = reader.fieldnames

        # Validamos las columnas columnas requeridas
        required_cols = {"nombre", "direccion", "latitud", "longitud", "ciudad", "estado", "pais", "codigo_postal"}
        missing_cols = required_cols - set(original_fieldnames)
        if missing_cols:
            raise ValueError(f"El CSV no contiene las siguientes columnas requeridas: {', '.join(missing_cols)}")

        # Nombres de las columnas
        fieldnames = [
            "id_nombre", "nombre", "direccion", "latitud", "longitud",
            "id_ciudad", "ciudad", "estado", "pais", "codigo_postal"
        ]

        # ordenamos y creamos el archivo de salida
        with open(output_path, mode='w', newline='', encoding='utf-8') as outfile:
            writer = csv.DictWriter(outfile, fieldnames=fieldnames)
            writer.writeheader()

            for row in reader:
                id_nombre = get_id(row["nombre"], nombres_dict)
                id_ciudad = get_id(row["ciudad"], ciudades_dict)

                new_row = {
                    "id_nombre": id_nombre,
                    "nombre": row["nombre"],
                    "direccion": row["direccion"],
                    "latitud": row["latitud"],
                    "longitud": row["longitud"],
                    "id_ciudad": id_ciudad,
                    "ciudad": row["ciudad"],
                    "estado": row["estado"],
                    "pais": row["pais"],
                    "codigo_postal": row["codigo_postal"]
                }
                writer.writerow(new_row)

process_file(input_path, output_path)

# Guardamos el catálogos actualizados 
save_catalog(nombres_dim_path, nombres_dict, "nombre", "id_nombre")
save_catalog(ciudades_dim_path, ciudades_dict, "ciudad", "id_ciudad")

print("Proceso completado con éxito.")

Proceso completado con éxito.


Pasamos el archivo a formato .parquet

In [67]:
# Cargamos el archivo CSV 
csv_path = r"H:/Nueva carpeta/overpass-turbo.eu/Parques_de_diversiones_Centros_comerciales_2016_final.csv"

# Ruta del archivo Parquet de salida
parquet_path = r"H:\Nueva carpeta\overpass-turbo.eu\Parques_de_diversiones_Centros_comerciales_2016_final.parquet"

df = pd.read_csv(csv_path)

# Guardamos el archivo Parquet
df.to_parquet(parquet_path, engine='pyarrow', index=False)

print(f"Archivo convertido exitosamente a Parquet: {parquet_path}")

Archivo convertido exitosamente a Parquet: H:\Nueva carpeta\overpass-turbo.eu\Parques_de_diversiones_Centros_comerciales_2016_final.parquet


Parques de diverciones Centros comerciales 2018

In [68]:
# 1. Cargamos los datos
file_path = r'H:/Nueva carpeta/overpass-turbo.eu/Parques de diverciones Centros comerciales 2016.csv'  
data = pd.read_csv(file_path, delimiter='\t')  

# 2. Realizamos una revisión general
print("Dimensiones del dataset:", data.shape)
print("\nNombres de las columnas:", data.columns.tolist())
print("\nPrimeras filas del dataset:")
print(data.head())

# 3. Mostramos la calidad de los datos
print("\nValores nulos por columna:")
print(data.isnull().sum())

print("\nTipos de datos:")
print(data.dtypes)

# 4. Realisaos una estadísticas descriptivas
print("\nEstadísticas descriptivas para columnas numéricas:")
print(data.describe())

print("\nValores únicos por columna:")
print(data.nunique())

Dimensiones del dataset: (348, 7)

Nombres de las columnas: ['name', 'addr:street', 'addr:city', 'addr:state', 'amenity', '@lat', '@lon']

Primeras filas del dataset:
                       name              addr:street      addr:city  \
0           Titusville Mall                      NaN            NaN   
1  Landmark Shopping Center                      NaN            NaN   
2            LandMark Plaza                      NaN            NaN   
3  Ellenton Premium Outlets  Factory Shops Boulevard       Ellenton   
4                  CocoWalk             Grand Avenue  Coconut Grove   

  addr:state  amenity       @lat       @lon  
0        NaN      NaN  28.575472 -80.802640  
1        NaN      NaN  28.068730 -82.436298  
2        NaN      NaN  28.068711 -82.437068  
3        NaN      NaN  27.534335 -82.506753  
4        NaN      NaN  25.728644 -80.242038  

Valores nulos por columna:
name           156
addr:street    247
addr:city      254
addr:state     269
amenity        348
@lat   

Desacrgamos datos geograficos mediante latitud y longitud

In [69]:
# Configuramos el geolocalizador con un user_agent único
geolocator = Nominatim(user_agent="mi_proyecto_geocodificacion_v1")

# Función para realizar la geocodificación inversa y buscar el nombre del lugar
def reverse_geocode(lat, lon):
    try:
        location = geolocator.reverse((lat, lon), timeout=10)
        if location:
            address = location.address
            city = location.raw.get('address', {}).get('city', None)
            name = location.raw.get('name', None) or location.raw.get('address', {}).get('attraction', None)
            return address, city, name
        else:
            return "Dirección no encontrada", None, None
    except GeocoderTimedOut:
        return "Tiempo de espera agotado", None, None
    except GeocoderInsufficientPrivileges:
        return "Bloqueado: Verificar API o User-Agent", None, None

# Limpieza inicial del archivo
input_file = "H:/Nueva carpeta/overpass-turbo.eu/Parques de diverciones Centros comerciales 2018.csv"
cleaned_file = "H:/Nueva carpeta/overpass-turbo.eu/Parques de diverciones Centros comerciales 20181.csv"

with open(input_file, 'r', encoding='utf-8') as infile, open(cleaned_file, 'w', encoding='utf-8') as outfile:
    for line in infile:
        if line.strip():  # Omitir líneas vacías
            outfile.write(line)

# Cargamos el archivo limpio
try:
    data = pd.read_csv(cleaned_file, sep='\t', encoding='utf-8')
except pd.errors.ParserError as e:
    print(f"Error al cargar el archivo: {e}")
    data = pd.read_csv(cleaned_file, sep=',', encoding='utf-8', on_bad_lines='skip')

# Añadimos  la columna 'name' si no existe
if 'name' not in data.columns:
    data['name'] = None

# Actualizamos las columnas addr:street, addr:city y name
def update_address(row):
    if pd.notna(row['@lat']) and pd.notna(row['@lon']):
        address, city, name = reverse_geocode(row['@lat'], row['@lon'])
        row['addr:street'] = address
        row['addr:city'] = city
        row['name'] = name
    return row

# Aplicar la función fila por fila con pausas para respetar los límites de solicitudes
for index, row in data.iterrows():
    data.loc[index] = update_address(row)
    time.sleep(2)

# Guardar los resultados
output_file = "H:/Nueva carpeta/overpass-turbo.eu/Parques_direcciones_actualizadas_2018.csv"
data.to_csv(output_file, index=False)

# Mostrar las primeras filas procesadas
print(data.head())

                       name  \
0           Titusville Mall   
1  Landmark Shopping Center   
2            LandMark Plaza   
3  Lakeview Shopping Center   
4       Carrollwood Commons   

                                         addr:street         addr:city  \
0  Titusville Mall, Narvaez Drive, Titusville, Br...        Titusville   
1  Landmark Shopping Center, North 20th Street, H...              None   
2  LandMark Plaza, East Fletcher Avenue, Hillsbor...              None   
3  Lakeview Shopping Center, Hartford Street Sout...  Saint Petersburg   
4  Carrollwood Commons, North Dale Mabry Highway,...              None   

  addr:state  amenity       @lat       @lon  
0        NaN      NaN  28.575472 -80.802640  
1        NaN      NaN  28.068730 -82.436298  
2        NaN      NaN  28.068711 -82.437068  
3        NaN      NaN  27.747511 -82.680299  
4        NaN      NaN  28.087608 -82.505597  


Extraemos datos de la columna direccion

In [ ]:
# Cargamos  los datos 
archivo = "H:/Nueva carpeta/overpass-turbo.eu/Parques_direcciones_actualizadas_2018.csv" 
data = pd.read_csv(archivo)

# Renombramos  las columnas
data.rename(columns={
    'addr:street': 'direccion',
    'amenity': 'tipo',
    '@lat': 'latitud',
    '@lon': 'longitud'
}, inplace=True)

# Dividimos la columna direccion para extraer los datos
direccion_split = data['direccion'].fillna('').str.split(',', expand=True)

# Creamos las nuebas colunas a usar
data['nombre'] = direccion_split[0].str.strip()

# Extraemos la direccion combinando todas las partes de la dirección hasta el estado
data['direccion'] = direccion_split.apply(
    lambda row: ', '.join(filter(None, [row[1].strip(), row[2].strip()])), axis=1
)

# Extraemos la ciudad siempre y cuando tenga la palabra County 
data['ciudad'] = direccion_split.apply(
    lambda row: next((x.strip() for x in row if 'County' in str(x)), 'No definido'), axis=1
)

# Extraemos el estado  "Florida"
data['estado'] = direccion_split.apply(lambda row: 'Florida', axis=1)

# Extraemos  y validamos los códigos postales 
data['codigo_postal'] = direccion_split.apply(
    lambda row: next((re.search(r'\b\d{5}\b', str(x)).group(0) for x in row if re.search(r'\b\d{5}\b', str(x))), 'No definido'), axis=1
)

# Saparamos el peis "United States"
data['pais'] = direccion_split.apply(lambda row: 'United States', axis=1)

# Reordenamos las columnas 
data = data[['nombre', 'direccion', 'ciudad', 'estado', 'pais', 'codigo_postal', 'longitud', 'latitud']]

# Eliminamos  duplicados
data.drop_duplicates(inplace=True)

# Guardamos el resultado
processed_file_path = 'H:/Nueva carpeta/overpass-turbo.eu/Parques_de_diversiones_Centros_comerciales_2018.csv'
data.to_csv(processed_file_path, index=False)

# Análisamos de datos vacíos y duplicados
vacios = data.isnull().sum()
duplicados = data.duplicated().sum()

print("Análisis de datos:")
print("Valores vacíos por columna:")
print(vacios)
print(f"Número de filas duplicadas: {duplicados}")
print(data.head())

Análisis de datos:
Valores vacíos por columna:
nombre           0
direccion        0
ciudad           0
estado           0
pais             0
codigo_postal    0
longitud         0
latitud          0
dtype: int64
Número de filas duplicadas: 0
                     nombre                                      direccion  \
0           Titusville Mall                      Narvaez Drive, Titusville   
1  Landmark Shopping Center         North 20th Street, Hillsborough County   
2            LandMark Plaza      East Fletcher Avenue, Hillsborough County   
3  Lakeview Shopping Center        Hartford Street South, Saint Petersburg   
4       Carrollwood Commons  North Dale Mabry Highway, Hillsborough County   

                ciudad   estado           pais codigo_postal   longitud  \
0       Brevard County  Florida  United States         32780 -80.802640   
1  Hillsborough County  Florida  United States         33613 -82.436298   
2  Hillsborough County  Florida  United States         33613 -82

Borramos la palabra County

In [71]:
# Eliminamos la palabra 'County' de las columnas 'direccion' y 'ciudad'
data['direccion'] = data['direccion'].str.replace(r'\bCounty\b', '', regex=True).str.strip()
data['ciudad'] = data['ciudad'].str.replace(r'\bCounty\b', '', regex=True).str.strip()
print(data[['direccion', 'ciudad']].head())

# Guardamos el resultado 
processed_file_path = 'H:/Nueva carpeta/overpass-turbo.eu/Parques_de_diversiones_Centros_comerciales_2018.csv'
data.to_csv(processed_file_path, index=False)


                                 direccion        ciudad
0                Narvaez Drive, Titusville       Brevard
1          North 20th Street, Hillsborough  Hillsborough
2       East Fletcher Avenue, Hillsborough  Hillsborough
3  Hartford Street South, Saint Petersburg      Pinellas
4   North Dale Mabry Highway, Hillsborough  Hillsborough


Creamos Id_nombre y id ciudad

In [72]:
5
# Cargamos las rutas de los archivos
input_path = r"H:/Nueva carpeta/overpass-turbo.eu/Parques_de_diversiones_Centros_comerciales_2018.csv"
output_path = r"H:/Nueva carpeta/overpass-turbo.eu/Parques_de_diversiones_Centros_comerciales_2018_final.csv"
nombres_dim_path = "H:/Nueva carpeta/overpass-turbo.eu/nombres_dim.csv"
ciudades_dim_path = "H:/Nueva carpeta/overpass-turbo.eu/ciudades_dim.csv"

# Cargamos los catálogos
def load_catalog(path, key, value):
    catalog = {}
    if os.path.exists(path):
        with open(path, mode='r', encoding='utf-8') as f:
            reader = csv.DictReader(f)
            for row in reader:
                catalog[row[key]] = int(row[value])
    return catalog

# Guardamos los catálogos de ciudad y nombres
def save_catalog(path, catalog, key, value):
    with open(path, mode='w', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=[value, key])
        writer.writeheader()
        for k, v in catalog.items():
            writer.writerow({value: v, key: k})

# Generamos IDs únicos
def get_id(value, catalog):
    if value not in catalog:
        catalog[value] = len(catalog) + 1
    return catalog[value]

# Cargamos catálogos existentes
nombres_dict = load_catalog(nombres_dim_path, "nombre", "id_nombre")
ciudades_dict = load_catalog(ciudades_dim_path, "ciudad", "id_ciudad")

# Procesamos archivo de entrada
def process_file(input_path, output_path):
    if not os.path.exists(input_path):
        raise FileNotFoundError(f"El archivo de entrada no existe: {input_path}")

    with open(input_path, mode='r', encoding='utf-8') as infile:
        reader = csv.DictReader(infile)
        original_fieldnames = reader.fieldnames

        # Validamos las columnas columnas requeridas
        required_cols = {"nombre", "direccion", "latitud", "longitud", "ciudad", "estado", "pais", "codigo_postal"}
        missing_cols = required_cols - set(original_fieldnames)
        if missing_cols:
            raise ValueError(f"El CSV no contiene las siguientes columnas requeridas: {', '.join(missing_cols)}")

        # Nombres de las columnas
        fieldnames = [
            "id_nombre", "nombre", "direccion", "latitud", "longitud",
            "id_ciudad", "ciudad", "estado", "pais", "codigo_postal"
        ]

        # ordenamos y creamos el archivo de salida
        with open(output_path, mode='w', newline='', encoding='utf-8') as outfile:
            writer = csv.DictWriter(outfile, fieldnames=fieldnames)
            writer.writeheader()

            for row in reader:
                id_nombre = get_id(row["nombre"], nombres_dict)
                id_ciudad = get_id(row["ciudad"], ciudades_dict)

                new_row = {
                    "id_nombre": id_nombre,
                    "nombre": row["nombre"],
                    "direccion": row["direccion"],
                    "latitud": row["latitud"],
                    "longitud": row["longitud"],
                    "id_ciudad": id_ciudad,
                    "ciudad": row["ciudad"],
                    "estado": row["estado"],
                    "pais": row["pais"],
                    "codigo_postal": row["codigo_postal"]
                }
                writer.writerow(new_row)

process_file(input_path, output_path)

# Guardamos el catálogos actualizados 
save_catalog(nombres_dim_path, nombres_dict, "nombre", "id_nombre")
save_catalog(ciudades_dim_path, ciudades_dict, "ciudad", "id_ciudad")

print("Proceso completado con éxito.")

Proceso completado con éxito.


Trasformamos a formato parquet 

In [75]:
# Cargamos el archivo CSV 
csv_path = r"H:/Nueva carpeta/overpass-turbo.eu/Parques_de_diversiones_Centros_comerciales_2018_final.csv"

# Ruta del archivo Parquet de salida
parquet_path = r"H:\Nueva carpeta\overpass-turbo.eu\Parques_de_diversiones_Centros_comerciales_2018_final.parquet"

df = pd.read_csv(csv_path)

# Guardamos el archivo Parquet
df.to_parquet(parquet_path, engine='pyarrow', index=False)

print(f"Archivo convertido exitosamente a Parquet: {parquet_path}")

Archivo convertido exitosamente a Parquet: H:\Nueva carpeta\overpass-turbo.eu\Parques_de_diversiones_Centros_comerciales_2018_final.parquet


Parques de diverciones Centros comerciales 2019

In [ ]:
# 1. Cargamos los datos
file_path = r'H:/Nueva carpeta/overpass-turbo.eu/Parques de diverciones Centros comerciales 2019.csv'  
data = pd.read_csv(file_path, delimiter='\t')  

# 2. Realizamos una revisión general
print("Dimensiones del dataset:", data.shape)
print("\nNombres de las columnas:", data.columns.tolist())
print("\nPrimeras filas del dataset:")
print(data.head())

# 3. Mostramos la calidad de los datos
print("\nValores nulos por columna:")
print(data.isnull().sum())

print("\nTipos de datos:")
print(data.dtypes)

# 4. Realisamos  estadísticas descriptivas
print("\nEstadísticas descriptivas para columnas numéricas:")
print(data.describe())

print("\nValores únicos por columna:")
print(data.nunique())

Dimensiones del dataset: (404, 7)

Nombres de las columnas: ['name', 'addr:street', 'addr:city', 'addr:state', 'amenity', '@lat', '@lon']

Primeras filas del dataset:
                       name addr:street addr:city addr:state  amenity  \
0           Titusville Mall         NaN       NaN        NaN      NaN   
1  Landmark Shopping Center         NaN       NaN        NaN      NaN   
2            LandMark Plaza         NaN       NaN        NaN      NaN   
3  Lakeview Shopping Center         NaN       NaN        NaN      NaN   
4       Carrollwood Commons         NaN       NaN        NaN      NaN   

        @lat       @lon  
0  28.575472 -80.802640  
1  28.068730 -82.436298  
2  28.068711 -82.437068  
3  27.747511 -82.680299  
4  28.087608 -82.505597  

Valores nulos por columna:
name           194
addr:street    301
addr:city      310
addr:state     325
amenity        404
@lat             0
@lon             0
dtype: int64

Tipos de datos:
name            object
addr:street     object
a

Descargamos datos faltantes en base ala longitud y latitud

In [77]:
# Configuramos el geolocalizador con un user_agent único
geolocator = Nominatim(user_agent="mi_proyecto_geocodificacion_v1")

# Función para realizar la geocodificación inversa y buscar el nombre del lugar
def reverse_geocode(lat, lon):
    try:
        location = geolocator.reverse((lat, lon), timeout=10)
        if location:
            address = location.address
            city = location.raw.get('address', {}).get('city', None)
            name = location.raw.get('name', None) or location.raw.get('address', {}).get('attraction', None)
            return address, city, name
        else:
            return "Dirección no encontrada", None, None
    except GeocoderTimedOut:
        return "Tiempo de espera agotado", None, None
    except GeocoderInsufficientPrivileges:
        return "Bloqueado: Verificar API o User-Agent", None, None

# Limpieza inicial del archivo
input_file = "H:/Nueva carpeta/overpass-turbo.eu/Parques de diverciones Centros comerciales 2019.csv"
cleaned_file = "H:/Nueva carpeta/overpass-turbo.eu/Parques de diverciones Centros comerciales 20161.csv"

with open(input_file, 'r', encoding='utf-8') as infile, open(cleaned_file, 'w', encoding='utf-8') as outfile:
    for line in infile:
        if line.strip():  # Omitir líneas vacías
            outfile.write(line)

# Cargamos el archivo limpio
try:
    data = pd.read_csv(cleaned_file, sep='\t', encoding='utf-8')
except pd.errors.ParserError as e:
    print(f"Error al cargar el archivo: {e}")
    data = pd.read_csv(cleaned_file, sep=',', encoding='utf-8', on_bad_lines='skip')

# Añadimos  la columna 'name' si no existe
if 'name' not in data.columns:
    data['name'] = None

# Actualizamos las columnas addr:street, addr:city y name
def update_address(row):
    if pd.notna(row['@lat']) and pd.notna(row['@lon']):
        address, city, name = reverse_geocode(row['@lat'], row['@lon'])
        row['addr:street'] = address
        row['addr:city'] = city
        row['name'] = name
    return row

# Aplicar la función fila por fila con pausas para respetar los límites de solicitudes
for index, row in data.iterrows():
    data.loc[index] = update_address(row)
    time.sleep(2)

# Guardar los resultados
output_file = "H:/Nueva carpeta/overpass-turbo.eu/Parques_direcciones_actualizadas_2019.csv"
data.to_csv(output_file, index=False)

# Mostrar las primeras filas procesadas
print(data.head())

                       name  \
0           Titusville Mall   
1  Landmark Shopping Center   
2            LandMark Plaza   
3  Lakeview Shopping Center   
4       Carrollwood Commons   

                                         addr:street         addr:city  \
0  Titusville Mall, Narvaez Drive, Titusville, Br...        Titusville   
1  Landmark Shopping Center, North 20th Street, H...              None   
2  LandMark Plaza, East Fletcher Avenue, Hillsbor...              None   
3  Lakeview Shopping Center, Hartford Street Sout...  Saint Petersburg   
4  Carrollwood Commons, North Dale Mabry Highway,...              None   

  addr:state  amenity       @lat       @lon  
0        NaN      NaN  28.575472 -80.802640  
1        NaN      NaN  28.068730 -82.436298  
2        NaN      NaN  28.068711 -82.437068  
3        NaN      NaN  27.747511 -82.680299  
4        NaN      NaN  28.087608 -82.505597  


Extraemos la imformacion de la coluna direcciones

In [ ]:
# Cargamos  los datos 
archivo = "H:/Nueva carpeta/overpass-turbo.eu/Parques_direcciones_actualizadas_2019.csv" 
data = pd.read_csv(archivo)

# Renombramos  las columnas
data.rename(columns={
    'addr:street': 'direccion',
    'amenity': 'tipo',
    '@lat': 'latitud',
    '@lon': 'longitud'
}, inplace=True)

# Dividimos la columna direccion para extraer los datos
direccion_split = data['direccion'].fillna('').str.split(',', expand=True)

# Creamos las nuebas colunas a usar
data['nombre'] = direccion_split[0].str.strip()

# Extraemos la direccion combinando todas las partes de la dirección hasta el estado
data['direccion'] = direccion_split.apply(
    lambda row: ', '.join(filter(None, [row[1].strip(), row[2].strip()])), axis=1
)

# Extraemos la ciudad siempre y cuando tenga la palabra County 
data['ciudad'] = direccion_split.apply(
    lambda row: next((x.strip() for x in row if 'County' in str(x)), 'No definido'), axis=1
)

# Extraemos el estado  "Florida"
data['estado'] = direccion_split.apply(lambda row: 'Florida', axis=1)

# Extraemos  y validamos los códigos postales 
data['codigo_postal'] = direccion_split.apply(
    lambda row: next((re.search(r'\b\d{5}\b', str(x)).group(0) for x in row if re.search(r'\b\d{5}\b', str(x))), 'No definido'), axis=1
)

# Saparamos el peis "United States"
data['pais'] = direccion_split.apply(lambda row: 'United States', axis=1)

# Reordenamos las columnas 
data = data[['nombre', 'direccion', 'ciudad', 'estado', 'pais', 'codigo_postal', 'longitud', 'latitud']]

# Eliminamos  duplicados
data.drop_duplicates(inplace=True)

# Guardamos el resultado
processed_file_path = 'H:/Nueva carpeta/overpass-turbo.eu/Parques_de_diversiones_Centros_comerciales_2019.csv'
data.to_csv(processed_file_path, index=False)

# Análisamos de datos vacíos y duplicados
vacios = data.isnull().sum()
duplicados = data.duplicated().sum()

print("Análisis de datos:")
print("Valores vacíos por columna:")
print(vacios)
print(f"Número de filas duplicadas: {duplicados}")
print(data.head())

Análisis de datos:
Valores vacíos por columna:
nombre           0
direccion        0
ciudad           0
estado           0
pais             0
codigo_postal    0
longitud         0
latitud          0
dtype: int64
Número de filas duplicadas: 0
                     nombre                                      direccion  \
0           Titusville Mall                      Narvaez Drive, Titusville   
1  Landmark Shopping Center         North 20th Street, Hillsborough County   
2            LandMark Plaza      East Fletcher Avenue, Hillsborough County   
3  Lakeview Shopping Center        Hartford Street South, Saint Petersburg   
4       Carrollwood Commons  North Dale Mabry Highway, Hillsborough County   

                ciudad   estado           pais codigo_postal   longitud  \
0       Brevard County  Florida  United States         32780 -80.802640   
1  Hillsborough County  Florida  United States         33613 -82.436298   
2  Hillsborough County  Florida  United States         33613 -82

Eliminamos la palabra County

In [96]:
# Eliminamos la palabra 'County' de las columnas 'direccion' y 'ciudad'
data['direccion'] = data['direccion'].str.replace(r'\bCounty\b', '', regex=True).str.strip()
data['ciudad'] = data['ciudad'].str.replace(r'\bCounty\b', '', regex=True).str.strip()
print(data[['direccion', 'ciudad']].head())

# Guardamos el resultado 
processed_file_path = 'H:/Nueva carpeta/overpass-turbo.eu/Parques_de_diversiones_Centros_comerciales_2019.csv'
data.to_csv(processed_file_path, index=False)

                                 direccion        ciudad
0                Narvaez Drive, Titusville       Brevard
1          North 20th Street, Hillsborough  Hillsborough
2       East Fletcher Avenue, Hillsborough  Hillsborough
3  Hartford Street South, Saint Petersburg      Pinellas
4   North Dale Mabry Highway, Hillsborough  Hillsborough


CReamos las columnas Id nombre e id ciudad

In [97]:
# Cargamos las rutas de los archivos
input_path = r"H:/Nueva carpeta/overpass-turbo.eu/Parques_de_diversiones_Centros_comerciales_2019.csv"
output_path = r"H:/Nueva carpeta/overpass-turbo.eu/Parques_de_diversiones_Centros_comerciales_2019_final.csv"
nombres_dim_path = "H:/Nueva carpeta/overpass-turbo.eu/nombres_dim.csv"
ciudades_dim_path = "H:/Nueva carpeta/overpass-turbo.eu/ciudades_dim.csv"

# Cargamos los catálogos
def load_catalog(path, key, value):
    catalog = {}
    if os.path.exists(path):
        with open(path, mode='r', encoding='utf-8') as f:
            reader = csv.DictReader(f)
            for row in reader:
                catalog[row[key]] = int(row[value])
    return catalog

# Guardamos los catálogos de ciudad y nombres
def save_catalog(path, catalog, key, value):
    with open(path, mode='w', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=[value, key])
        writer.writeheader()
        for k, v in catalog.items():
            writer.writerow({value: v, key: k})

# Generamos IDs únicos
def get_id(value, catalog):
    if value not in catalog:
        catalog[value] = len(catalog) + 1
    return catalog[value]

# Cargamos catálogos existentes
nombres_dict = load_catalog(nombres_dim_path, "nombre", "id_nombre")
ciudades_dict = load_catalog(ciudades_dim_path, "ciudad", "id_ciudad")

# Procesamos archivo de entrada
def process_file(input_path, output_path):
    if not os.path.exists(input_path):
        raise FileNotFoundError(f"El archivo de entrada no existe: {input_path}")

    with open(input_path, mode='r', encoding='utf-8') as infile:
        reader = csv.DictReader(infile)
        original_fieldnames = reader.fieldnames

        # Validamos las columnas columnas requeridas
        required_cols = {"nombre", "direccion", "latitud", "longitud", "ciudad", "estado", "pais", "codigo_postal"}
        missing_cols = required_cols - set(original_fieldnames)
        if missing_cols:
            raise ValueError(f"El CSV no contiene las siguientes columnas requeridas: {', '.join(missing_cols)}")

        # Nombres de las columnas
        fieldnames = [
            "id_nombre", "nombre", "direccion", "latitud", "longitud",
            "id_ciudad", "ciudad", "estado", "pais", "codigo_postal"
        ]

        # ordenamos y creamos el archivo de salida
        with open(output_path, mode='w', newline='', encoding='utf-8') as outfile:
            writer = csv.DictWriter(outfile, fieldnames=fieldnames)
            writer.writeheader()

            for row in reader:
                id_nombre = get_id(row["nombre"], nombres_dict)
                id_ciudad = get_id(row["ciudad"], ciudades_dict)

                new_row = {
                    "id_nombre": id_nombre,
                    "nombre": row["nombre"],
                    "direccion": row["direccion"],
                    "latitud": row["latitud"],
                    "longitud": row["longitud"],
                    "id_ciudad": id_ciudad,
                    "ciudad": row["ciudad"],
                    "estado": row["estado"],
                    "pais": row["pais"],
                    "codigo_postal": row["codigo_postal"]
                }
                writer.writerow(new_row)

process_file(input_path, output_path)

# Guardamos el catálogos actualizados 
save_catalog(nombres_dim_path, nombres_dict, "nombre", "id_nombre")
save_catalog(ciudades_dim_path, ciudades_dict, "ciudad", "id_ciudad")

print("Proceso completado con éxito.")

Proceso completado con éxito.


Pasamos a formato parquet

In [98]:
# Cargamos el archivo CSV 
csv_path = r"H:/Nueva carpeta/overpass-turbo.eu/Parques_de_diversiones_Centros_comerciales_2019_final.csv"

# Ruta del archivo Parquet de salida
parquet_path = r"H:\Nueva carpeta\overpass-turbo.eu\Parques_de_diversiones_Centros_comerciales_2019_final.parquet"

df = pd.read_csv(csv_path)

# Guardamos el archivo Parquet
df.to_parquet(parquet_path, engine='pyarrow', index=False)

print(f"Archivo convertido exitosamente a Parquet: {parquet_path}")

Archivo convertido exitosamente a Parquet: H:\Nueva carpeta\overpass-turbo.eu\Parques_de_diversiones_Centros_comerciales_2019_final.parquet


Parques de diverciones Centros comerciales 2020

In [99]:
# 1. Cargamos los datos
file_path = r'H:/Nueva carpeta/overpass-turbo.eu/Parques de diverciones Centros comerciales 2020.csv'  
data = pd.read_csv(file_path, delimiter='\t')  

# 2. Realizamos una revisión general
print("Dimensiones del dataset:", data.shape)
print("\nNombres de las columnas:", data.columns.tolist())
print("\nPrimeras filas del dataset:")
print(data.head())

# 3. Mostramos la calidad de los datos
print("\nValores nulos por columna:")
print(data.isnull().sum())

print("\nTipos de datos:")
print(data.dtypes)

# 4. Realisaos una estadísticas descriptivas
print("\nEstadísticas descriptivas para columnas numéricas:")
print(data.describe())

print("\nValores únicos por columna:")
print(data.nunique())

Dimensiones del dataset: (404, 7)

Nombres de las columnas: ['name', 'addr:street', 'addr:city', 'addr:state', 'amenity', '@lat', '@lon']

Primeras filas del dataset:
                       name addr:street addr:city addr:state  amenity  \
0           Titusville Mall         NaN       NaN        NaN      NaN   
1  Landmark Shopping Center         NaN       NaN        NaN      NaN   
2            LandMark Plaza         NaN       NaN        NaN      NaN   
3  Lakeview Shopping Center         NaN       NaN        NaN      NaN   
4       Carrollwood Commons         NaN       NaN        NaN      NaN   

        @lat       @lon  
0  28.575472 -80.802640  
1  28.068730 -82.436298  
2  28.068711 -82.437068  
3  27.747511 -82.680299  
4  28.087608 -82.505597  

Valores nulos por columna:
name           194
addr:street    301
addr:city      310
addr:state     325
amenity        404
@lat             0
@lon             0
dtype: int64

Tipos de datos:
name            object
addr:street     object
a

Descargamos informacion faltante en base a la longuitud y latitud

In [100]:
# Configuramos el geolocalizador con un user_agent único
geolocator = Nominatim(user_agent="mi_proyecto_geocodificacion_v1")

# Función para realizar la geocodificación inversa y buscar el nombre del lugar
def reverse_geocode(lat, lon):
    try:
        location = geolocator.reverse((lat, lon), timeout=10)
        if location:
            address = location.address
            city = location.raw.get('address', {}).get('city', None)
            name = location.raw.get('name', None) or location.raw.get('address', {}).get('attraction', None)
            return address, city, name
        else:
            return "Dirección no encontrada", None, None
    except GeocoderTimedOut:
        return "Tiempo de espera agotado", None, None
    except GeocoderInsufficientPrivileges:
        return "Bloqueado: Verificar API o User-Agent", None, None

# Limpieza inicial del archivo
input_file = "H:/Nueva carpeta/overpass-turbo.eu/Parques de diverciones Centros comerciales 2020.csv"
cleaned_file = "H:/Nueva carpeta/overpass-turbo.eu/Parques de diverciones Centros comerciales 20161.csv"

with open(input_file, 'r', encoding='utf-8') as infile, open(cleaned_file, 'w', encoding='utf-8') as outfile:
    for line in infile:
        if line.strip():  # Omitir líneas vacías
            outfile.write(line)

# Cargamos el archivo limpio
try:
    data = pd.read_csv(cleaned_file, sep='\t', encoding='utf-8')
except pd.errors.ParserError as e:
    print(f"Error al cargar el archivo: {e}")
    data = pd.read_csv(cleaned_file, sep=',', encoding='utf-8', on_bad_lines='skip')

# Añadimos  la columna 'name' si no existe
if 'name' not in data.columns:
    data['name'] = None

# Actualizamos las columnas addr:street, addr:city y name
def update_address(row):
    if pd.notna(row['@lat']) and pd.notna(row['@lon']):
        address, city, name = reverse_geocode(row['@lat'], row['@lon'])
        row['addr:street'] = address
        row['addr:city'] = city
        row['name'] = name
    return row

# Aplicar la función fila por fila con pausas para respetar los límites de solicitudes
for index, row in data.iterrows():
    data.loc[index] = update_address(row)
    time.sleep(2)

# Guardar los resultados
output_file = "H:/Nueva carpeta/overpass-turbo.eu/Parques_direcciones_actualizadas_2020.csv"
data.to_csv(output_file, index=False)

# Mostrar las primeras filas procesadas
print(data.head())

                       name  \
0           Titusville Mall   
1  Landmark Shopping Center   
2            LandMark Plaza   
3  Lakeview Shopping Center   
4       Carrollwood Commons   

                                         addr:street         addr:city  \
0  Titusville Mall, Narvaez Drive, Titusville, Br...        Titusville   
1  Landmark Shopping Center, North 20th Street, H...              None   
2  LandMark Plaza, East Fletcher Avenue, Hillsbor...              None   
3  Lakeview Shopping Center, Hartford Street Sout...  Saint Petersburg   
4  Carrollwood Commons, North Dale Mabry Highway,...              None   

  addr:state  amenity       @lat       @lon  
0        NaN      NaN  28.575472 -80.802640  
1        NaN      NaN  28.068730 -82.436298  
2        NaN      NaN  28.068711 -82.437068  
3        NaN      NaN  27.747511 -82.680299  
4        NaN      NaN  28.087608 -82.505597  


Cambiamos nombres de columnas y eliminamos las menos relebantes

In [ ]:
3
# Cargamos  los datos 
archivo = "H:/Nueva carpeta/overpass-turbo.eu/Parques_direcciones_actualizadas_2020.csv"  
data = pd.read_csv(archivo)

# Renombramos  las columnas
data.rename(columns={
    'addr:street': 'direccion',
    'amenity': 'tipo',
    '@lat': 'latitud',
    '@lon': 'longitud'
}, inplace=True)

# Dividimos la columna direccion para extraer los datos
direccion_split = data['direccion'].fillna('').str.split(',', expand=True)

# Creamos las nuebas colunas a usar
data['nombre'] = direccion_split[0].str.strip()

# Extraemos la direccion combinando todas las partes de la dirección hasta el estado
data['direccion'] = direccion_split.apply(
    lambda row: ', '.join(filter(None, [row[1].strip(), row[2].strip()])), axis=1
)

# Extraemos la ciudad siempre y cuando tenga la palabra County 
data['ciudad'] = direccion_split.apply(
    lambda row: next((x.strip() for x in row if 'County' in str(x)), 'No definido'), axis=1
)

# Extraemos el estado  "Florida"
data['estado'] = direccion_split.apply(lambda row: 'Florida', axis=1)

# Extraemos  y validamos los códigos postales 
data['codigo_postal'] = direccion_split.apply(
    lambda row: next((re.search(r'\b\d{5}\b', str(x)).group(0) for x in row if re.search(r'\b\d{5}\b', str(x))), 'No definido'), axis=1
)

# Saparamos el peis "United States"
data['pais'] = direccion_split.apply(lambda row: 'United States', axis=1)

# Reordenamos las columnas 
data = data[['nombre', 'direccion', 'ciudad', 'estado', 'pais', 'codigo_postal', 'longitud', 'latitud']]

# Eliminamos  duplicados
data.drop_duplicates(inplace=True)

# Guardamos el resultado
processed_file_path = 'H:/Nueva carpeta/overpass-turbo.eu/Parques_de_diversiones_Centros_comerciales_2020.csv'
data.to_csv(processed_file_path, index=False)

# Análisamos de datos vacíos y duplicados
vacios = data.isnull().sum()
duplicados = data.duplicated().sum()

print("Análisis de datos:")
print("Valores vacíos por columna:")
print(vacios)
print(f"Número de filas duplicadas: {duplicados}")
print(data.head())

Análisis de datos:
Valores vacíos por columna:
nombre           0
direccion        0
ciudad           0
estado           0
pais             0
codigo_postal    0
longitud         0
latitud          0
dtype: int64
Número de filas duplicadas: 0
                     nombre                                      direccion  \
0           Titusville Mall                      Narvaez Drive, Titusville   
1  Landmark Shopping Center         North 20th Street, Hillsborough County   
2            LandMark Plaza      East Fletcher Avenue, Hillsborough County   
3  Lakeview Shopping Center        Hartford Street South, Saint Petersburg   
4       Carrollwood Commons  North Dale Mabry Highway, Hillsborough County   

                ciudad   estado           pais codigo_postal   longitud  \
0       Brevard County  Florida  United States         32780 -80.802640   
1  Hillsborough County  Florida  United States         33613 -82.436298   
2  Hillsborough County  Florida  United States         33613 -82

Boramos la palabra county

In [102]:
# Eliminamos la palabra 'County' de las columnas 'direccion' y 'ciudad'
data['direccion'] = data['direccion'].str.replace(r'\bCounty\b', '', regex=True).str.strip()
data['ciudad'] = data['ciudad'].str.replace(r'\bCounty\b', '', regex=True).str.strip()
print(data[['direccion', 'ciudad']].head())

# Guardamos el resultado 
processed_file_path = 'H:/Nueva carpeta/overpass-turbo.eu/Parques_de_diversiones_Centros_comerciales_2020.csv'
data.to_csv(processed_file_path, index=False)

                                 direccion        ciudad
0                Narvaez Drive, Titusville       Brevard
1          North 20th Street, Hillsborough  Hillsborough
2       East Fletcher Avenue, Hillsborough  Hillsborough
3  Hartford Street South, Saint Petersburg      Pinellas
4   North Dale Mabry Highway, Hillsborough  Hillsborough


Creamos la columna Id nombre i Id ciudad

In [103]:
# Cargamos las rutas de los archivos
input_path = r"H:/Nueva carpeta/overpass-turbo.eu/Parques_de_diversiones_Centros_comerciales_2020.csv"
output_path = r"H:/Nueva carpeta/overpass-turbo.eu/Parques_de_diversiones_Centros_comerciales_2020_final.csv"
nombres_dim_path = "H:/Nueva carpeta/overpass-turbo.eu/nombres_dim.csv"
ciudades_dim_path = "H:/Nueva carpeta/overpass-turbo.eu/ciudades_dim.csv"

# Cargamos los catálogos
def load_catalog(path, key, value):
    catalog = {}
    if os.path.exists(path):
        with open(path, mode='r', encoding='utf-8') as f:
            reader = csv.DictReader(f)
            for row in reader:
                catalog[row[key]] = int(row[value])
    return catalog

# Guardamos los catálogos de ciudad y nombres
def save_catalog(path, catalog, key, value):
    with open(path, mode='w', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=[value, key])
        writer.writeheader()
        for k, v in catalog.items():
            writer.writerow({value: v, key: k})

# Generamos IDs únicos
def get_id(value, catalog):
    if value not in catalog:
        catalog[value] = len(catalog) + 1
    return catalog[value]

# Cargamos catálogos existentes
nombres_dict = load_catalog(nombres_dim_path, "nombre", "id_nombre")
ciudades_dict = load_catalog(ciudades_dim_path, "ciudad", "id_ciudad")

# Procesamos archivo de entrada
def process_file(input_path, output_path):
    if not os.path.exists(input_path):
        raise FileNotFoundError(f"El archivo de entrada no existe: {input_path}")

    with open(input_path, mode='r', encoding='utf-8') as infile:
        reader = csv.DictReader(infile)
        original_fieldnames = reader.fieldnames

        # Validamos las columnas columnas requeridas
        required_cols = {"nombre", "direccion", "latitud", "longitud", "ciudad", "estado", "pais", "codigo_postal"}
        missing_cols = required_cols - set(original_fieldnames)
        if missing_cols:
            raise ValueError(f"El CSV no contiene las siguientes columnas requeridas: {', '.join(missing_cols)}")

        # Nombres de las columnas
        fieldnames = [
            "id_nombre", "nombre", "direccion", "latitud", "longitud",
            "id_ciudad", "ciudad", "estado", "pais", "codigo_postal"
        ]

        # ordenamos y creamos el archivo de salida
        with open(output_path, mode='w', newline='', encoding='utf-8') as outfile:
            writer = csv.DictWriter(outfile, fieldnames=fieldnames)
            writer.writeheader()

            for row in reader:
                id_nombre = get_id(row["nombre"], nombres_dict)
                id_ciudad = get_id(row["ciudad"], ciudades_dict)

                new_row = {
                    "id_nombre": id_nombre,
                    "nombre": row["nombre"],
                    "direccion": row["direccion"],
                    "latitud": row["latitud"],
                    "longitud": row["longitud"],
                    "id_ciudad": id_ciudad,
                    "ciudad": row["ciudad"],
                    "estado": row["estado"],
                    "pais": row["pais"],
                    "codigo_postal": row["codigo_postal"]
                }
                writer.writerow(new_row)

process_file(input_path, output_path)

# Guardamos el catálogos actualizados 
save_catalog(nombres_dim_path, nombres_dict, "nombre", "id_nombre")
save_catalog(ciudades_dim_path, ciudades_dict, "ciudad", "id_ciudad")

print("Proceso completado con éxito.")

Proceso completado con éxito.


Combertimos a formato parquet

In [104]:
# Cargamos el archivo CSV 
csv_path = r"H:/Nueva carpeta/overpass-turbo.eu/Parques_de_diversiones_Centros_comerciales_2020_final.csv"

# Ruta del archivo Parquet de salida
parquet_path = r"H:\Nueva carpeta\overpass-turbo.eu\Parques_de_diversiones_Centros_comerciales_2020_final.parquet"

df = pd.read_csv(csv_path)

# Guardamos el archivo Parquet
df.to_parquet(parquet_path, engine='pyarrow', index=False)

print(f"Archivo convertido exitosamente a Parquet: {parquet_path}")

Archivo convertido exitosamente a Parquet: H:\Nueva carpeta\overpass-turbo.eu\Parques_de_diversiones_Centros_comerciales_2020_final.parquet


Parques de diverciones Centros comerciales 2021

In [ ]:
# 1. Cargamos los datos
file_path = r'H:/Nueva carpeta/overpass-turbo.eu/Parques de diverciones Centros comerciales 2021.csv'  
data = pd.read_csv(file_path, delimiter='\t')  

# 2. Realizamos una revisión general
print("Dimensiones del dataset:", data.shape)
print("\nNombres de las columnas:", data.columns.tolist())
print("\nPrimeras filas del dataset:")
print(data.head())

# 3. Mostramos la calidad de los datos
print("\nValores nulos por columna:")
print(data.isnull().sum())

print("\nTipos de datos:")
print(data.dtypes)

# 4. Realisaos una estadísticas descriptivas
print("\nEstadísticas descriptivas para columnas numéricas:")
print(data.describe())

print("\nValores únicos por columna:")
print(data.nunique())

Dimensiones del dataset: (404, 7)

Nombres de las columnas: ['name', 'addr:street', 'addr:city', 'addr:state', 'amenity', '@lat', '@lon']

Primeras filas del dataset:
                       name addr:street addr:city addr:state  amenity  \
0           Titusville Mall         NaN       NaN        NaN      NaN   
1  Landmark Shopping Center         NaN       NaN        NaN      NaN   
2            LandMark Plaza         NaN       NaN        NaN      NaN   
3  Lakeview Shopping Center         NaN       NaN        NaN      NaN   
4       Carrollwood Commons         NaN       NaN        NaN      NaN   

        @lat       @lon  
0  28.575472 -80.802640  
1  28.068730 -82.436298  
2  28.068711 -82.437068  
3  27.747511 -82.680299  
4  28.087608 -82.505597  

Valores nulos por columna:
name           194
addr:street    301
addr:city      310
addr:state     325
amenity        404
@lat             0
@lon             0
dtype: int64

Tipos de datos:
name            object
addr:street     object
a

Descargamos los datos que nos faltan en base a longitud y latitud 

In [106]:
# Configuramos el geolocalizador con un user_agent único
geolocator = Nominatim(user_agent="mi_proyecto_geocodificacion_v1")

# Función para realizar la geocodificación inversa y buscar el nombre del lugar
def reverse_geocode(lat, lon):
    try:
        location = geolocator.reverse((lat, lon), timeout=10)
        if location:
            address = location.address
            city = location.raw.get('address', {}).get('city', None)
            name = location.raw.get('name', None) or location.raw.get('address', {}).get('attraction', None)
            return address, city, name
        else:
            return "Dirección no encontrada", None, None
    except GeocoderTimedOut:
        return "Tiempo de espera agotado", None, None
    except GeocoderInsufficientPrivileges:
        return "Bloqueado: Verificar API o User-Agent", None, None

# Limpieza inicial del archivo
input_file = "H:/Nueva carpeta/overpass-turbo.eu/Parques de diverciones Centros comerciales 2021.csv"
cleaned_file = "H:/Nueva carpeta/overpass-turbo.eu/Parques de diverciones Centros comerciales 20161.csv"

with open(input_file, 'r', encoding='utf-8') as infile, open(cleaned_file, 'w', encoding='utf-8') as outfile:
    for line in infile:
        if line.strip():  # Omitir líneas vacías
            outfile.write(line)

# Cargamos el archivo limpio
try:
    data = pd.read_csv(cleaned_file, sep='\t', encoding='utf-8')
except pd.errors.ParserError as e:
    print(f"Error al cargar el archivo: {e}")
    data = pd.read_csv(cleaned_file, sep=',', encoding='utf-8', on_bad_lines='skip')

# Añadimos  la columna 'name' si no existe
if 'name' not in data.columns:
    data['name'] = None

# Actualizamos las columnas addr:street, addr:city y name
def update_address(row):
    if pd.notna(row['@lat']) and pd.notna(row['@lon']):
        address, city, name = reverse_geocode(row['@lat'], row['@lon'])
        row['addr:street'] = address
        row['addr:city'] = city
        row['name'] = name
    return row

# Aplicar la función fila por fila con pausas para respetar los límites de solicitudes
for index, row in data.iterrows():
    data.loc[index] = update_address(row)
    time.sleep(2)

# Guardar los resultados
output_file = "H:/Nueva carpeta/overpass-turbo.eu/Parques_direcciones_actualizadas_2021.csv"
data.to_csv(output_file, index=False)

# Mostrar las primeras filas procesadas
print(data.head())

                       name  \
0           Titusville Mall   
1  Landmark Shopping Center   
2            LandMark Plaza   
3  Lakeview Shopping Center   
4       Carrollwood Commons   

                                         addr:street         addr:city  \
0  Titusville Mall, Narvaez Drive, Titusville, Br...        Titusville   
1  Landmark Shopping Center, North 20th Street, H...              None   
2  LandMark Plaza, East Fletcher Avenue, Hillsbor...              None   
3  Lakeview Shopping Center, Hartford Street Sout...  Saint Petersburg   
4  Carrollwood Commons, North Dale Mabry Highway,...              None   

  addr:state  amenity       @lat       @lon  
0        NaN      NaN  28.575472 -80.802640  
1        NaN      NaN  28.068730 -82.436298  
2        NaN      NaN  28.068711 -82.437068  
3        NaN      NaN  27.747511 -82.680299  
4        NaN      NaN  28.087608 -82.505597  


Extraemos la direccion de la columna direccion

In [107]:
# Cargamos  los datos 
archivo = "H:/Nueva carpeta/overpass-turbo.eu/Parques_direcciones_actualizadas_2021.csv"  

# Renombramos  las columnas
data.rename(columns={
    'addr:street': 'direccion',
    'amenity': 'tipo',
    '@lat': 'latitud',
    '@lon': 'longitud'
}, inplace=True)

# Dividimos la columna direccion para extraer los datos
direccion_split = data['direccion'].fillna('').str.split(',', expand=True)

# Creamos las nuebas colunas a usar
data['nombre'] = direccion_split[0].str.strip()

# Extraemos la direccion combinando todas las partes de la dirección hasta el estado
data['direccion'] = direccion_split.apply(
    lambda row: ', '.join(filter(None, [row[1].strip(), row[2].strip()])), axis=1
)

# Extraemos la ciudad siempre y cuando tenga la palabra County 
data['ciudad'] = direccion_split.apply(
    lambda row: next((x.strip() for x in row if 'County' in str(x)), 'No definido'), axis=1
)

# Extraemos el estado  "Florida"
data['estado'] = direccion_split.apply(lambda row: 'Florida', axis=1)

# Extraemos  y validamos los códigos postales 
data['codigo_postal'] = direccion_split.apply(
    lambda row: next((re.search(r'\b\d{5}\b', str(x)).group(0) for x in row if re.search(r'\b\d{5}\b', str(x))), 'No definido'), axis=1
)

# Saparamos el peis "United States"
data['pais'] = direccion_split.apply(lambda row: 'United States', axis=1)

# Reordenamos las columnas 
data = data[['nombre', 'direccion', 'ciudad', 'estado', 'pais', 'codigo_postal', 'longitud', 'latitud']]

# Eliminamos  duplicados
data.drop_duplicates(inplace=True)

# Guardamos el resultado
processed_file_path = 'H:/Nueva carpeta/overpass-turbo.eu/Parques_de_diversiones_Centros_comerciales_2021.csv'
data.to_csv(processed_file_path, index=False)

# Análisamos de datos vacíos y duplicados
vacios = data.isnull().sum()
duplicados = data.duplicated().sum()

print("Análisis de datos:")
print("Valores vacíos por columna:")
print(vacios)
print(f"Número de filas duplicadas: {duplicados}")
print(data.head())

Análisis de datos:
Valores vacíos por columna:
nombre           0
direccion        0
ciudad           0
estado           0
pais             0
codigo_postal    0
longitud         0
latitud          0
dtype: int64
Número de filas duplicadas: 0
                     nombre                                      direccion  \
0           Titusville Mall                      Narvaez Drive, Titusville   
1  Landmark Shopping Center         North 20th Street, Hillsborough County   
2            LandMark Plaza      East Fletcher Avenue, Hillsborough County   
3  Lakeview Shopping Center        Hartford Street South, Saint Petersburg   
4       Carrollwood Commons  North Dale Mabry Highway, Hillsborough County   

                ciudad   estado           pais codigo_postal   longitud  \
0       Brevard County  Florida  United States         32780 -80.802640   
1  Hillsborough County  Florida  United States         33613 -82.436298   
2  Hillsborough County  Florida  United States         33613 -82

Eliminamos la palabra County

In [108]:
# Eliminamos la palabra 'County' de las columnas 'direccion' y 'ciudad'
data['direccion'] = data['direccion'].str.replace(r'\bCounty\b', '', regex=True).str.strip()
data['ciudad'] = data['ciudad'].str.replace(r'\bCounty\b', '', regex=True).str.strip()
print(data[['direccion', 'ciudad']].head())

# Guardamos el resultado 
processed_file_path = 'H:/Nueva carpeta/overpass-turbo.eu/Parques_de_diversiones_Centros_comerciales_2021.csv'
data.to_csv(processed_file_path, index=False)

                                 direccion        ciudad
0                Narvaez Drive, Titusville       Brevard
1          North 20th Street, Hillsborough  Hillsborough
2       East Fletcher Avenue, Hillsborough  Hillsborough
3  Hartford Street South, Saint Petersburg      Pinellas
4   North Dale Mabry Highway, Hillsborough  Hillsborough


Creamos la columna id nombre y id ciudad

In [109]:
# Cargamos las rutas de los archivos
input_path = r"H:/Nueva carpeta/overpass-turbo.eu/Parques_de_diversiones_Centros_comerciales_2021.csv"
output_path = r"H:/Nueva carpeta/overpass-turbo.eu/Parques_de_diversiones_Centros_comerciales_2021_final.csv"
nombres_dim_path = "H:/Nueva carpeta/overpass-turbo.eu/nombres_dim.csv"
ciudades_dim_path = "H:/Nueva carpeta/overpass-turbo.eu/ciudades_dim.csv"

# Cargamos los catálogos
def load_catalog(path, key, value):
    catalog = {}
    if os.path.exists(path):
        with open(path, mode='r', encoding='utf-8') as f:
            reader = csv.DictReader(f)
            for row in reader:
                catalog[row[key]] = int(row[value])
    return catalog

# Guardamos los catálogos de ciudad y nombres
def save_catalog(path, catalog, key, value):
    with open(path, mode='w', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=[value, key])
        writer.writeheader()
        for k, v in catalog.items():
            writer.writerow({value: v, key: k})

# Generamos IDs únicos
def get_id(value, catalog):
    if value not in catalog:
        catalog[value] = len(catalog) + 1
    return catalog[value]

# Cargamos catálogos existentes
nombres_dict = load_catalog(nombres_dim_path, "nombre", "id_nombre")
ciudades_dict = load_catalog(ciudades_dim_path, "ciudad", "id_ciudad")

# Procesamos archivo de entrada
def process_file(input_path, output_path):
    if not os.path.exists(input_path):
        raise FileNotFoundError(f"El archivo de entrada no existe: {input_path}")

    with open(input_path, mode='r', encoding='utf-8') as infile:
        reader = csv.DictReader(infile)
        original_fieldnames = reader.fieldnames

        # Validamos las columnas columnas requeridas
        required_cols = {"nombre", "direccion", "latitud", "longitud", "ciudad", "estado", "pais", "codigo_postal"}
        missing_cols = required_cols - set(original_fieldnames)
        if missing_cols:
            raise ValueError(f"El CSV no contiene las siguientes columnas requeridas: {', '.join(missing_cols)}")

        # Nombres de las columnas
        fieldnames = [
            "id_nombre", "nombre", "direccion", "latitud", "longitud",
            "id_ciudad", "ciudad", "estado", "pais", "codigo_postal"
        ]

        # ordenamos y creamos el archivo de salida
        with open(output_path, mode='w', newline='', encoding='utf-8') as outfile:
            writer = csv.DictWriter(outfile, fieldnames=fieldnames)
            writer.writeheader()

            for row in reader:
                id_nombre = get_id(row["nombre"], nombres_dict)
                id_ciudad = get_id(row["ciudad"], ciudades_dict)

                new_row = {
                    "id_nombre": id_nombre,
                    "nombre": row["nombre"],
                    "direccion": row["direccion"],
                    "latitud": row["latitud"],
                    "longitud": row["longitud"],
                    "id_ciudad": id_ciudad,
                    "ciudad": row["ciudad"],
                    "estado": row["estado"],
                    "pais": row["pais"],
                    "codigo_postal": row["codigo_postal"]
                }
                writer.writerow(new_row)

process_file(input_path, output_path)

# Guardamos el catálogos actualizados 
save_catalog(nombres_dim_path, nombres_dict, "nombre", "id_nombre")
save_catalog(ciudades_dim_path, ciudades_dict, "ciudad", "id_ciudad")

print("Proceso completado con éxito.")

Proceso completado con éxito.


Pasamos a formato parquet 

In [110]:
# Cargamos el archivo CSV 
csv_path = r"H:/Nueva carpeta/overpass-turbo.eu/Parques_de_diversiones_Centros_comerciales_2021_final.csv"

# Ruta del archivo Parquet de salida
parquet_path = r"H:\Nueva carpeta\overpass-turbo.eu\Parques_de_diversiones_Centros_comerciales_2021_final.parquet"

df = pd.read_csv(csv_path)

# Guardamos el archivo Parquet
df.to_parquet(parquet_path, engine='pyarrow', index=False)

print(f"Archivo convertido exitosamente a Parquet: {parquet_path}")

Archivo convertido exitosamente a Parquet: H:\Nueva carpeta\overpass-turbo.eu\Parques_de_diversiones_Centros_comerciales_2021_final.parquet


Parques de diverciones Centros comerciales 2022

In [14]:
# 1. Cargamos los datos
file_path = r'H:/Nueva carpeta/overpass-turbo.eu/Parques de diverciones Centros comerciales 2022.csv'  
data = pd.read_csv(file_path, delimiter='\t')  

# 2. Realizamos una revisión general
print("Dimensiones del dataset:", data.shape)
print("\nNombres de las columnas:", data.columns.tolist())
print("\nPrimeras filas del dataset:")
print(data.head())

# 3. Mostramos la calidad de los datos
print("\nValores nulos por columna:")
print(data.isnull().sum())

print("\nTipos de datos:")
print(data.dtypes)

# 4. Realisaos una estadísticas descriptivas
print("\nEstadísticas descriptivas para columnas numéricas:")
print(data.describe())

print("\nValores únicos por columna:")
print(data.nunique())

Dimensiones del dataset: (404, 7)

Nombres de las columnas: ['name', 'addr:street', 'addr:city', 'addr:state', 'amenity', '@lat', '@lon']

Primeras filas del dataset:
                       name addr:street addr:city addr:state  amenity  \
0           Titusville Mall         NaN       NaN        NaN      NaN   
1  Landmark Shopping Center         NaN       NaN        NaN      NaN   
2            LandMark Plaza         NaN       NaN        NaN      NaN   
3  Lakeview Shopping Center         NaN       NaN        NaN      NaN   
4       Carrollwood Commons         NaN       NaN        NaN      NaN   

        @lat       @lon  
0  28.575472 -80.802640  
1  28.068730 -82.436298  
2  28.068711 -82.437068  
3  27.747511 -82.680299  
4  28.087608 -82.505597  

Valores nulos por columna:
name           194
addr:street    301
addr:city      310
addr:state     325
amenity        404
@lat             0
@lon             0
dtype: int64

Tipos de datos:
name            object
addr:street     object
a

Extraemos datos faltantes en base a la longitud y latitud

In [15]:
# Configuramos el geolocalizador con un user_agent único
geolocator = Nominatim(user_agent="mi_proyecto_geocodificacion_v1")

# Función para realizar la geocodificación inversa y buscar el nombre del lugar
def reverse_geocode(lat, lon):
    try:
        location = geolocator.reverse((lat, lon), timeout=10)
        if location:
            address = location.address
            city = location.raw.get('address', {}).get('city', None)
            name = location.raw.get('name', None) or location.raw.get('address', {}).get('attraction', None)
            return address, city, name
        else:
            return "Dirección no encontrada", None, None
    except GeocoderTimedOut:
        return "Tiempo de espera agotado", None, None
    except GeocoderInsufficientPrivileges:
        return "Bloqueado: Verificar API o User-Agent", None, None

# Limpieza inicial del archivo
input_file = "H:/Nueva carpeta/overpass-turbo.eu/Parques de diverciones Centros comerciales 2022.csv"
cleaned_file = "H:/Nueva carpeta/overpass-turbo.eu/Parques de diverciones Centros comerciales 20161.csv"

with open(input_file, 'r', encoding='utf-8') as infile, open(cleaned_file, 'w', encoding='utf-8') as outfile:
    for line in infile:
        if line.strip():  # Omitir líneas vacías
            outfile.write(line)

# Cargamos el archivo limpio
try:
    data = pd.read_csv(cleaned_file, sep='\t', encoding='utf-8')
except pd.errors.ParserError as e:
    print(f"Error al cargar el archivo: {e}")
    data = pd.read_csv(cleaned_file, sep=',', encoding='utf-8', on_bad_lines='skip')

# Añadimos  la columna 'name' si no existe
if 'name' not in data.columns:
    data['name'] = None

# Actualizamos las columnas addr:street, addr:city y name
def update_address(row):
    if pd.notna(row['@lat']) and pd.notna(row['@lon']):
        address, city, name = reverse_geocode(row['@lat'], row['@lon'])
        row['addr:street'] = address
        row['addr:city'] = city
        row['name'] = name
    return row

# Aplicar la función fila por fila con pausas para respetar los límites de solicitudes
for index, row in data.iterrows():
    data.loc[index] = update_address(row)
    time.sleep(2)

# Guardar los resultados
output_file = "H:/Nueva carpeta/overpass-turbo.eu/Parques_direcciones_actualizadas_2022.csv"
data.to_csv(output_file, index=False)

# Mostrar las primeras filas procesadas
print(data.head())

                       name  \
0           Titusville Mall   
1  Landmark Shopping Center   
2            LandMark Plaza   
3  Lakeview Shopping Center   
4       Carrollwood Commons   

                                         addr:street         addr:city  \
0  Titusville Mall, Narvaez Drive, Titusville, Br...        Titusville   
1  Landmark Shopping Center, North 20th Street, H...              None   
2  LandMark Plaza, East Fletcher Avenue, Hillsbor...              None   
3  Lakeview Shopping Center, Hartford Street Sout...  Saint Petersburg   
4  Carrollwood Commons, North Dale Mabry Highway,...              None   

  addr:state  amenity       @lat       @lon  
0        NaN      NaN  28.575472 -80.802640  
1        NaN      NaN  28.068730 -82.436298  
2        NaN      NaN  28.068711 -82.437068  
3        NaN      NaN  27.747511 -82.680299  
4        NaN      NaN  28.087608 -82.505597  


Se extrae los datos de la columna direcciones

In [16]:
# Cargamos  los datos 
archivo = "H:/Nueva carpeta/overpass-turbo.eu/Parques_direcciones_actualizadas_2022.csv"  
data = pd.read_csv(archivo)

# Renombramos  las columnas
data.rename(columns={
    'addr:street': 'direccion',
    'amenity': 'tipo',
    '@lat': 'latitud',
    '@lon': 'longitud'
}, inplace=True)

# Dividimos la columna direccion para extraer los datos
direccion_split = data['direccion'].fillna('').str.split(',', expand=True)

# Creamos las nuebas colunas a usar
data['nombre'] = direccion_split[0].str.strip()

# Extraemos la direccion combinando todas las partes de la dirección hasta el estado
data['direccion'] = direccion_split.apply(
    lambda row: ', '.join(filter(None, [row[1].strip(), row[2].strip()])), axis=1
)

# Extraemos la ciudad siempre y cuando tenga la palabra County 
data['ciudad'] = direccion_split.apply(
    lambda row: next((x.strip() for x in row if 'County' in str(x)), 'No definido'), axis=1
)

# Extraemos el estado  "Florida"
data['estado'] = direccion_split.apply(lambda row: 'Florida', axis=1)

# Extraemos  y validamos los códigos postales 
data['codigo_postal'] = direccion_split.apply(
    lambda row: next((re.search(r'\b\d{5}\b', str(x)).group(0) for x in row if re.search(r'\b\d{5}\b', str(x))), 'No definido'), axis=1
)

# Saparamos el peis "United States"
data['pais'] = direccion_split.apply(lambda row: 'United States', axis=1)

# Reordenamos las columnas 
data = data[['nombre', 'direccion', 'ciudad', 'estado', 'pais', 'codigo_postal', 'longitud', 'latitud']]

# Eliminamos  duplicados
data.drop_duplicates(inplace=True)

# Guardamos el resultado
processed_file_path = 'H:/Nueva carpeta/overpass-turbo.eu/Parques_de_diversiones_Centros_comerciales_2023.csv'
data.to_csv(processed_file_path, index=False)

# Análisamos de datos vacíos y duplicados
vacios = data.isnull().sum()
duplicados = data.duplicated().sum()

print("Análisis de datos:")
print("Valores vacíos por columna:")
print(vacios)
print(f"Número de filas duplicadas: {duplicados}")
print(data.head())

Análisis de datos:
Valores vacíos por columna:
nombre           0
direccion        0
ciudad           0
estado           0
pais             0
codigo_postal    0
longitud         0
latitud          0
dtype: int64
Número de filas duplicadas: 0
                     nombre                                      direccion  \
0           Titusville Mall                      Narvaez Drive, Titusville   
1  Landmark Shopping Center         North 20th Street, Hillsborough County   
2            LandMark Plaza      East Fletcher Avenue, Hillsborough County   
3  Lakeview Shopping Center        Hartford Street South, Saint Petersburg   
4       Carrollwood Commons  North Dale Mabry Highway, Hillsborough County   

                ciudad   estado           pais codigo_postal   longitud  \
0       Brevard County  Florida  United States         32780 -80.802640   
1  Hillsborough County  Florida  United States         33613 -82.436298   
2  Hillsborough County  Florida  United States         33613 -82

Eliminamos la palabra County

In [17]:
# Eliminamos la palabra 'County' de las columnas 'direccion' y 'ciudad'
data['direccion'] = data['direccion'].str.replace(r'\bCounty\b', '', regex=True).str.strip()
data['ciudad'] = data['ciudad'].str.replace(r'\bCounty\b', '', regex=True).str.strip()
print(data[['direccion', 'ciudad']].head())

# Guardamos el resultado 
processed_file_path = 'H:/Nueva carpeta/overpass-turbo.eu/Parques_de_diversiones_Centros_comerciales_2022.csv'
data.to_csv(processed_file_path, index=False)

                                 direccion        ciudad
0                Narvaez Drive, Titusville       Brevard
1          North 20th Street, Hillsborough  Hillsborough
2       East Fletcher Avenue, Hillsborough  Hillsborough
3  Hartford Street South, Saint Petersburg      Pinellas
4   North Dale Mabry Highway, Hillsborough  Hillsborough


Creamps las columnas id nombres y id ciudad 

In [18]:
# Cargamos las rutas de los archivos
input_path = r"H:/Nueva carpeta/overpass-turbo.eu/Parques_de_diversiones_Centros_comerciales_2022.csv"
output_path = r"H:/Nueva carpeta/overpass-turbo.eu/Parques_de_diversiones_Centros_comerciales_2022_final.csv"
nombres_dim_path = "H:/Nueva carpeta/overpass-turbo.eu/nombres_dim.csv"
ciudades_dim_path = "H:/Nueva carpeta/overpass-turbo.eu/ciudades_dim.csv"

# Cargamos los catálogos
def load_catalog(path, key, value):
    catalog = {}
    if os.path.exists(path):
        with open(path, mode='r', encoding='utf-8') as f:
            reader = csv.DictReader(f)
            for row in reader:
                catalog[row[key]] = int(row[value])
    return catalog

# Guardamos los catálogos de ciudad y nombres
def save_catalog(path, catalog, key, value):
    with open(path, mode='w', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=[value, key])
        writer.writeheader()
        for k, v in catalog.items():
            writer.writerow({value: v, key: k})

# Generamos IDs únicos
def get_id(value, catalog):
    if value not in catalog:
        catalog[value] = len(catalog) + 1
    return catalog[value]

# Cargamos catálogos existentes
nombres_dict = load_catalog(nombres_dim_path, "nombre", "id_nombre")
ciudades_dict = load_catalog(ciudades_dim_path, "ciudad", "id_ciudad")

# Procesamos archivo de entrada
def process_file(input_path, output_path):
    if not os.path.exists(input_path):
        raise FileNotFoundError(f"El archivo de entrada no existe: {input_path}")

    with open(input_path, mode='r', encoding='utf-8') as infile:
        reader = csv.DictReader(infile)
        original_fieldnames = reader.fieldnames

        # Validamos las columnas columnas requeridas
        required_cols = {"nombre", "direccion", "latitud", "longitud", "ciudad", "estado", "pais", "codigo_postal"}
        missing_cols = required_cols - set(original_fieldnames)
        if missing_cols:
            raise ValueError(f"El CSV no contiene las siguientes columnas requeridas: {', '.join(missing_cols)}")

        # Nombres de las columnas
        fieldnames = [
            "id_nombre", "nombre", "direccion", "latitud", "longitud",
            "id_ciudad", "ciudad", "estado", "pais", "codigo_postal"
        ]

        # ordenamos y creamos el archivo de salida
        with open(output_path, mode='w', newline='', encoding='utf-8') as outfile:
            writer = csv.DictWriter(outfile, fieldnames=fieldnames)
            writer.writeheader()

            for row in reader:
                id_nombre = get_id(row["nombre"], nombres_dict)
                id_ciudad = get_id(row["ciudad"], ciudades_dict)

                new_row = {
                    "id_nombre": id_nombre,
                    "nombre": row["nombre"],
                    "direccion": row["direccion"],
                    "latitud": row["latitud"],
                    "longitud": row["longitud"],
                    "id_ciudad": id_ciudad,
                    "ciudad": row["ciudad"],
                    "estado": row["estado"],
                    "pais": row["pais"],
                    "codigo_postal": row["codigo_postal"]
                }
                writer.writerow(new_row)

process_file(input_path, output_path)

# Guardamos el catálogos actualizados 
save_catalog(nombres_dim_path, nombres_dict, "nombre", "id_nombre")
save_catalog(ciudades_dim_path, ciudades_dict, "ciudad", "id_ciudad")

print("Proceso completado con éxito.")

Proceso completado con éxito.


Pasamos el archivo a formato .parquet 

In [19]:
# Cargamos el archivo CSV 
csv_path = r"H:/Nueva carpeta/overpass-turbo.eu/Parques_de_diversiones_Centros_comerciales_2022_final.csv"

# Ruta del archivo Parquet de salida
parquet_path = r"H:\Nueva carpeta\overpass-turbo.eu\Parques_de_diversiones_Centros_comerciales_2022_final.parquet"

df = pd.read_csv(csv_path)

# Guardamos el archivo Parquet
df.to_parquet(parquet_path, engine='pyarrow', index=False)

print(f"Archivo convertido exitosamente a Parquet: {parquet_path}")

Archivo convertido exitosamente a Parquet: H:\Nueva carpeta\overpass-turbo.eu\Parques_de_diversiones_Centros_comerciales_2022_final.parquet


Parques de diverciones Centros comerciales 2023

In [20]:
# 1. Cargamos los datos
file_path = r'H:/Nueva carpeta/overpass-turbo.eu/Parques de diverciones Centros comerciales 2023.csv'  
data = pd.read_csv(file_path, delimiter='\t')  

# 2. Realizamos una revisión general
print("Dimensiones del dataset:", data.shape)
print("\nNombres de las columnas:", data.columns.tolist())
print("\nPrimeras filas del dataset:")
print(data.head())

# 3. Mostramos la calidad de los datos
print("\nValores nulos por columna:")
print(data.isnull().sum())

print("\nTipos de datos:")
print(data.dtypes)

# 4. Realisaos una estadísticas descriptivas
print("\nEstadísticas descriptivas para columnas numéricas:")
print(data.describe())

print("\nValores únicos por columna:")
print(data.nunique())

Dimensiones del dataset: (404, 7)

Nombres de las columnas: ['name', 'addr:street', 'addr:city', 'addr:state', 'amenity', '@lat', '@lon']

Primeras filas del dataset:
                       name addr:street addr:city addr:state  amenity  \
0           Titusville Mall         NaN       NaN        NaN      NaN   
1  Landmark Shopping Center         NaN       NaN        NaN      NaN   
2            LandMark Plaza         NaN       NaN        NaN      NaN   
3  Lakeview Shopping Center         NaN       NaN        NaN      NaN   
4       Carrollwood Commons         NaN       NaN        NaN      NaN   

        @lat       @lon  
0  28.575472 -80.802640  
1  28.068730 -82.436298  
2  28.068711 -82.437068  
3  27.747511 -82.680299  
4  28.087608 -82.505597  

Valores nulos por columna:
name           194
addr:street    301
addr:city      310
addr:state     325
amenity        404
@lat             0
@lon             0
dtype: int64

Tipos de datos:
name            object
addr:street     object
a

Extraemos datos nesesarios basados en la longitud y latitud 

In [21]:
# Configuramos el geolocalizador con un user_agent único
geolocator = Nominatim(user_agent="mi_proyecto_geocodificacion_v1")

# Función para realizar la geocodificación inversa y buscar el nombre del lugar
def reverse_geocode(lat, lon):
    try:
        location = geolocator.reverse((lat, lon), timeout=10)
        if location:
            address = location.address
            city = location.raw.get('address', {}).get('city', None)
            name = location.raw.get('name', None) or location.raw.get('address', {}).get('attraction', None)
            return address, city, name
        else:
            return "Dirección no encontrada", None, None
    except GeocoderTimedOut:
        return "Tiempo de espera agotado", None, None
    except GeocoderInsufficientPrivileges:
        return "Bloqueado: Verificar API o User-Agent", None, None

# Limpieza inicial del archivo
input_file = "H:/Nueva carpeta/overpass-turbo.eu/Parques de diverciones Centros comerciales 2023.csv"
cleaned_file = "H:/Nueva carpeta/overpass-turbo.eu/Parques de diverciones Centros comerciales 20161.csv"

with open(input_file, 'r', encoding='utf-8') as infile, open(cleaned_file, 'w', encoding='utf-8') as outfile:
    for line in infile:
        if line.strip():  # Omitir líneas vacías
            outfile.write(line)

# Cargamos el archivo limpio
try:
    data = pd.read_csv(cleaned_file, sep='\t', encoding='utf-8')
except pd.errors.ParserError as e:
    print(f"Error al cargar el archivo: {e}")
    data = pd.read_csv(cleaned_file, sep=',', encoding='utf-8', on_bad_lines='skip')

# Añadimos  la columna 'name' si no existe
if 'name' not in data.columns:
    data['name'] = None

# Actualizamos las columnas addr:street, addr:city y name
def update_address(row):
    if pd.notna(row['@lat']) and pd.notna(row['@lon']):
        address, city, name = reverse_geocode(row['@lat'], row['@lon'])
        row['addr:street'] = address
        row['addr:city'] = city
        row['name'] = name
    return row

# Aplicar la función fila por fila con pausas para respetar los límites de solicitudes
for index, row in data.iterrows():
    data.loc[index] = update_address(row)
    time.sleep(2)

# Guardar los resultados
output_file = "H:/Nueva carpeta/overpass-turbo.eu/Parques_direcciones_actualizadas_2023.csv"
data.to_csv(output_file, index=False)

# Mostrar las primeras filas procesadas
print(data.head())

                       name  \
0           Titusville Mall   
1  Landmark Shopping Center   
2            LandMark Plaza   
3  Lakeview Shopping Center   
4       Carrollwood Commons   

                                         addr:street         addr:city  \
0  Titusville Mall, Narvaez Drive, Titusville, Br...        Titusville   
1  Landmark Shopping Center, North 20th Street, H...              None   
2  LandMark Plaza, East Fletcher Avenue, Hillsbor...              None   
3  Lakeview Shopping Center, Hartford Street Sout...  Saint Petersburg   
4  Carrollwood Commons, North Dale Mabry Highway,...              None   

  addr:state  amenity       @lat       @lon  
0        NaN      NaN  28.575472 -80.802640  
1        NaN      NaN  28.068730 -82.436298  
2        NaN      NaN  28.068711 -82.437068  
3        NaN      NaN  27.747511 -82.680299  
4        NaN      NaN  28.087608 -82.505597  


Eliminamos columnas poco relebantes 

In [22]:
# Cargamos  los datos 
archivo = "H:/Nueva carpeta/overpass-turbo.eu/Parques_direcciones_actualizadas_2023.csv" 
data = pd.read_csv(archivo)

# Renombramos  las columnas
data.rename(columns={
    'addr:street': 'direccion',
    'amenity': 'tipo',
    '@lat': 'latitud',
    '@lon': 'longitud'
}, inplace=True)

# Dividimos la columna direccion para extraer los datos
direccion_split = data['direccion'].fillna('').str.split(',', expand=True)

# Creamos las nuebas colunas a usar
data['nombre'] = direccion_split[0].str.strip()

# Extraemos la direccion combinando todas las partes de la dirección hasta el estado
data['direccion'] = direccion_split.apply(
    lambda row: ', '.join(filter(None, [row[1].strip(), row[2].strip()])), axis=1
)

# Extraemos la ciudad siempre y cuando tenga la palabra County 
data['ciudad'] = direccion_split.apply(
    lambda row: next((x.strip() for x in row if 'County' in str(x)), 'No definido'), axis=1
)

# Extraemos el estado  "Florida"
data['estado'] = direccion_split.apply(lambda row: 'Florida', axis=1)

# Extraemos  y validamos los códigos postales 
data['codigo_postal'] = direccion_split.apply(
    lambda row: next((re.search(r'\b\d{5}\b', str(x)).group(0) for x in row if re.search(r'\b\d{5}\b', str(x))), 'No definido'), axis=1
)

# Saparamos el peis "United States"
data['pais'] = direccion_split.apply(lambda row: 'United States', axis=1)

# Reordenamos las columnas 
data = data[['nombre', 'direccion', 'ciudad', 'estado', 'pais', 'codigo_postal', 'longitud', 'latitud']]

# Eliminamos  duplicados
data.drop_duplicates(inplace=True)

# Guardamos el resultado
processed_file_path = 'H:/Nueva carpeta/overpass-turbo.eu/Parques_de_diversiones_Centros_comerciales_2023.csv'
data.to_csv(processed_file_path, index=False)

# Análisamos de datos vacíos y duplicados
vacios = data.isnull().sum()
duplicados = data.duplicated().sum()

print("Análisis de datos:")
print("Valores vacíos por columna:")
print(vacios)
print(f"Número de filas duplicadas: {duplicados}")
print(data.head())

Análisis de datos:
Valores vacíos por columna:
nombre           0
direccion        0
ciudad           0
estado           0
pais             0
codigo_postal    0
longitud         0
latitud          0
dtype: int64
Número de filas duplicadas: 0
                     nombre                                      direccion  \
0           Titusville Mall                      Narvaez Drive, Titusville   
1  Landmark Shopping Center         North 20th Street, Hillsborough County   
2            LandMark Plaza      East Fletcher Avenue, Hillsborough County   
3  Lakeview Shopping Center        Hartford Street South, Saint Petersburg   
4       Carrollwood Commons  North Dale Mabry Highway, Hillsborough County   

                ciudad   estado           pais codigo_postal   longitud  \
0       Brevard County  Florida  United States         32780 -80.802640   
1  Hillsborough County  Florida  United States         33613 -82.436298   
2  Hillsborough County  Florida  United States         33613 -82

Eliminamos la palabra county 

In [23]:
# Eliminamos la palabra 'County' de las columnas 'direccion' y 'ciudad'
data['direccion'] = data['direccion'].str.replace(r'\bCounty\b', '', regex=True).str.strip()
data['ciudad'] = data['ciudad'].str.replace(r'\bCounty\b', '', regex=True).str.strip()
print(data[['direccion', 'ciudad']].head())

# Guardamos el resultado 
processed_file_path = 'H:/Nueva carpeta/overpass-turbo.eu/Parques_de_diversiones_Centros_comerciales_2023.csv'
data.to_csv(processed_file_path, index=False)

                                 direccion        ciudad
0                Narvaez Drive, Titusville       Brevard
1          North 20th Street, Hillsborough  Hillsborough
2       East Fletcher Avenue, Hillsborough  Hillsborough
3  Hartford Street South, Saint Petersburg      Pinellas
4   North Dale Mabry Highway, Hillsborough  Hillsborough


Creamos la columna id nombre y id ciudad

In [24]:
5
# Cargamos las rutas de los archivos
input_path = r"H:/Nueva carpeta/overpass-turbo.eu/Parques_de_diversiones_Centros_comerciales_2023.csv"
output_path = r"H:/Nueva carpeta/overpass-turbo.eu/Parques_de_diversiones_Centros_comerciales_2023_final.csv"
nombres_dim_path = "H:/Nueva carpeta/overpass-turbo.eu/nombres_dim.csv"
ciudades_dim_path = "H:/Nueva carpeta/overpass-turbo.eu/ciudades_dim.csv"

# Cargamos los catálogos
def load_catalog(path, key, value):
    catalog = {}
    if os.path.exists(path):
        with open(path, mode='r', encoding='utf-8') as f:
            reader = csv.DictReader(f)
            for row in reader:
                catalog[row[key]] = int(row[value])
    return catalog

# Guardamos los catálogos de ciudad y nombres
def save_catalog(path, catalog, key, value):
    with open(path, mode='w', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=[value, key])
        writer.writeheader()
        for k, v in catalog.items():
            writer.writerow({value: v, key: k})

# Generamos IDs únicos
def get_id(value, catalog):
    if value not in catalog:
        catalog[value] = len(catalog) + 1
    return catalog[value]

# Cargamos catálogos existentes
nombres_dict = load_catalog(nombres_dim_path, "nombre", "id_nombre")
ciudades_dict = load_catalog(ciudades_dim_path, "ciudad", "id_ciudad")

# Procesamos archivo de entrada
def process_file(input_path, output_path):
    if not os.path.exists(input_path):
        raise FileNotFoundError(f"El archivo de entrada no existe: {input_path}")

    with open(input_path, mode='r', encoding='utf-8') as infile:
        reader = csv.DictReader(infile)
        original_fieldnames = reader.fieldnames

        # Validamos las columnas columnas requeridas
        required_cols = {"nombre", "direccion", "latitud", "longitud", "ciudad", "estado", "pais", "codigo_postal"}
        missing_cols = required_cols - set(original_fieldnames)
        if missing_cols:
            raise ValueError(f"El CSV no contiene las siguientes columnas requeridas: {', '.join(missing_cols)}")

        # Nombres de las columnas
        fieldnames = [
            "id_nombre", "nombre", "direccion", "latitud", "longitud",
            "id_ciudad", "ciudad", "estado", "pais", "codigo_postal"
        ]

        # ordenamos y creamos el archivo de salida
        with open(output_path, mode='w', newline='', encoding='utf-8') as outfile:
            writer = csv.DictWriter(outfile, fieldnames=fieldnames)
            writer.writeheader()

            for row in reader:
                id_nombre = get_id(row["nombre"], nombres_dict)
                id_ciudad = get_id(row["ciudad"], ciudades_dict)

                new_row = {
                    "id_nombre": id_nombre,
                    "nombre": row["nombre"],
                    "direccion": row["direccion"],
                    "latitud": row["latitud"],
                    "longitud": row["longitud"],
                    "id_ciudad": id_ciudad,
                    "ciudad": row["ciudad"],
                    "estado": row["estado"],
                    "pais": row["pais"],
                    "codigo_postal": row["codigo_postal"]
                }
                writer.writerow(new_row)

process_file(input_path, output_path)

# Guardamos el catálogos actualizados 
save_catalog(nombres_dim_path, nombres_dict, "nombre", "id_nombre")
save_catalog(ciudades_dim_path, ciudades_dict, "ciudad", "id_ciudad")

print("Proceso completado con éxito.")

Proceso completado con éxito.


Pasasmos el archivo a formato parquet 

In [25]:
# Cargamos el archivo CSV 
csv_path = r"H:/Nueva carpeta/overpass-turbo.eu/Parques_de_diversiones_Centros_comerciales_2023_final.csv"

# Ruta del archivo Parquet de salida
parquet_path = r"H:\Nueva carpeta\overpass-turbo.eu\Parques_de_diversiones_Centros_comerciales_2023_final.parquet"

df = pd.read_csv(csv_path)

# Guardamos el archivo Parquet
df.to_parquet(parquet_path, engine='pyarrow', index=False)

print(f"Archivo convertido exitosamente a Parquet: {parquet_path}")

Archivo convertido exitosamente a Parquet: H:\Nueva carpeta\overpass-turbo.eu\Parques_de_diversiones_Centros_comerciales_2023_final.parquet


Parques de diverciones Centros comerciales 2024

In [12]:
# 1. Cargamos los datos
file_path = r'H:/Nueva carpeta/overpass-turbo.eu/Parques de diverciones Centros comerciales 2024.csv'  
data = pd.read_csv(file_path, delimiter='\t')  

# 2. Realizamos una revisión general
print("Dimensiones del dataset:", data.shape)
print("\nNombres de las columnas:", data.columns.tolist())
print("\nPrimeras filas del dataset:")
print(data.head())

# 3. Mostramos la calidad de los datos
print("\nValores nulos por columna:")
print(data.isnull().sum())

print("\nTipos de datos:")
print(data.dtypes)

# 4. Realisaos una estadísticas descriptivas
print("\nEstadísticas descriptivas para columnas numéricas:")
print(data.describe())

print("\nValores únicos por columna:")
print(data.nunique())

Dimensiones del dataset: (404, 8)

Nombres de las columnas: ['name', 'addr:street', 'addr:city', 'addr:state', 'addr:postcode', 'amenity', '@lat', '@lon']

Primeras filas del dataset:
                       name addr:street addr:city addr:state addr:postcode  \
0           Titusville Mall         NaN       NaN        NaN           NaN   
1  Landmark Shopping Center         NaN       NaN        NaN           NaN   
2            LandMark Plaza         NaN       NaN        NaN           NaN   
3  Lakeview Shopping Center         NaN       NaN        NaN           NaN   
4       Carrollwood Commons         NaN       NaN        NaN           NaN   

   amenity       @lat       @lon  
0      NaN  28.575472 -80.802640  
1      NaN  28.068730 -82.436298  
2      NaN  28.068711 -82.437068  
3      NaN  27.747511 -82.680299  
4      NaN  28.087608 -82.505597  

Valores nulos por columna:
name             194
addr:street      301
addr:city        310
addr:state       325
addr:postcode    311
amen

Extraemos los datos faltantes en base ala latitud y longitud 

In [13]:
# Configuramos el geolocalizador con un user_agent único
geolocator = Nominatim(user_agent="mi_proyecto_geocodificacion_v1")

# Función para realizar la geocodificación inversa y buscar el nombre del lugar
def reverse_geocode(lat, lon):
    try:
        location = geolocator.reverse((lat, lon), timeout=10)
        if location:
            address = location.address
            city = location.raw.get('address', {}).get('city', None)
            name = location.raw.get('name', None) or location.raw.get('address', {}).get('attraction', None)
            return address, city, name
        else:
            return "Dirección no encontrada", None, None
    except GeocoderTimedOut:
        return "Tiempo de espera agotado", None, None
    except GeocoderInsufficientPrivileges:
        return "Bloqueado: Verificar API o User-Agent", None, None

# Limpieza inicial del archivo
input_file = "H:/Nueva carpeta/overpass-turbo.eu/Parques de diverciones Centros comerciales 2024.csv"
cleaned_file = "H:/Nueva carpeta/overpass-turbo.eu/Parques de diverciones Centros comerciales 20161.csv"

with open(input_file, 'r', encoding='utf-8') as infile, open(cleaned_file, 'w', encoding='utf-8') as outfile:
    for line in infile:
        if line.strip():  # Omitir líneas vacías
            outfile.write(line)

# Cargamos el archivo limpio
try:
    data = pd.read_csv(cleaned_file, sep='\t', encoding='utf-8')
except pd.errors.ParserError as e:
    print(f"Error al cargar el archivo: {e}")
    data = pd.read_csv(cleaned_file, sep=',', encoding='utf-8', on_bad_lines='skip')

# Añadimos  la columna 'name' si no existe
if 'name' not in data.columns:
    data['name'] = None

# Actualizamos las columnas addr:street, addr:city y name
def update_address(row):
    if pd.notna(row['@lat']) and pd.notna(row['@lon']):
        address, city, name = reverse_geocode(row['@lat'], row['@lon'])
        row['addr:street'] = address
        row['addr:city'] = city
        row['name'] = name
    return row

# Aplicar la función fila por fila con pausas para respetar los límites de solicitudes
for index, row in data.iterrows():
    data.loc[index] = update_address(row)
    time.sleep(2)

# Guardar los resultados
output_file = "H:/Nueva carpeta/overpass-turbo.eu/Parques_direcciones_actualizadas_2024.csv"
data.to_csv(output_file, index=False)

# Mostrar las primeras filas procesadas
print(data.head())

                       name  \
0           Titusville Mall   
1  Landmark Shopping Center   
2            LandMark Plaza   
3  Lakeview Shopping Center   
4       Carrollwood Commons   

                                         addr:street         addr:city  \
0  Titusville Mall, Narvaez Drive, Titusville, Br...        Titusville   
1  Landmark Shopping Center, North 20th Street, H...              None   
2  LandMark Plaza, East Fletcher Avenue, Hillsbor...              None   
3  Lakeview Shopping Center, Hartford Street Sout...  Saint Petersburg   
4  Carrollwood Commons, North Dale Mabry Highway,...              None   

  addr:state addr:postcode  amenity       @lat       @lon  
0        NaN           NaN      NaN  28.575472 -80.802640  
1        NaN           NaN      NaN  28.068730 -82.436298  
2        NaN           NaN      NaN  28.068711 -82.437068  
3        NaN           NaN      NaN  27.747511 -82.680299  
4        NaN           NaN      NaN  28.087608 -82.505597  


Eliminamos colunas innecesarias y cambiamos los nombres

In [26]:
# Cargamos  los datos 
archivo = "H:/Nueva carpeta/overpass-turbo.eu/Parques_direcciones_actualizadas_2024.csv" 
data = pd.read_csv(archivo)

# Renombramos  las columnas
data.rename(columns={
    'addr:street': 'direccion',
    'amenity': 'tipo',
    '@lat': 'latitud',
    '@lon': 'longitud'
}, inplace=True)

# Dividimos la columna direccion para extraer los datos
direccion_split = data['direccion'].fillna('').str.split(',', expand=True)

# Creamos las nuebas colunas a usar
data['nombre'] = direccion_split[0].str.strip()

# Extraemos la direccion combinando todas las partes de la dirección hasta el estado
data['direccion'] = direccion_split.apply(
    lambda row: ', '.join(filter(None, [row[1].strip(), row[2].strip()])), axis=1
)

# Extraemos la ciudad siempre y cuando tenga la palabra County 
data['ciudad'] = direccion_split.apply(
    lambda row: next((x.strip() for x in row if 'County' in str(x)), 'No definido'), axis=1
)

# Extraemos el estado  "Florida"
data['estado'] = direccion_split.apply(lambda row: 'Florida', axis=1)

# Extraemos  y validamos los códigos postales 
data['codigo_postal'] = direccion_split.apply(
    lambda row: next((re.search(r'\b\d{5}\b', str(x)).group(0) for x in row if re.search(r'\b\d{5}\b', str(x))), 'No definido'), axis=1
)

# Saparamos el peis "United States"
data['pais'] = direccion_split.apply(lambda row: 'United States', axis=1)

# Reordenamos las columnas 
data = data[['nombre', 'direccion', 'ciudad', 'estado', 'pais', 'codigo_postal', 'longitud', 'latitud']]

# Eliminamos  duplicados
data.drop_duplicates(inplace=True)

# Guardamos el resultado
processed_file_path = 'H:/Nueva carpeta/overpass-turbo.eu/Parques_de_diversiones_Centros_comerciales_2023.csv'
data.to_csv(processed_file_path, index=False)

# Análisamos de datos vacíos y duplicados
vacios = data.isnull().sum()
duplicados = data.duplicated().sum()

print("Análisis de datos:")
print("Valores vacíos por columna:")
print(vacios)
print(f"Número de filas duplicadas: {duplicados}")
print(data.head())

Análisis de datos:
Valores vacíos por columna:
nombre           0
direccion        0
ciudad           0
estado           0
pais             0
codigo_postal    0
longitud         0
latitud          0
dtype: int64
Número de filas duplicadas: 0
                     nombre                                      direccion  \
0           Titusville Mall                      Narvaez Drive, Titusville   
1  Landmark Shopping Center         North 20th Street, Hillsborough County   
2            LandMark Plaza      East Fletcher Avenue, Hillsborough County   
3  Lakeview Shopping Center        Hartford Street South, Saint Petersburg   
4       Carrollwood Commons  North Dale Mabry Highway, Hillsborough County   

                ciudad   estado           pais codigo_postal   longitud  \
0       Brevard County  Florida  United States         32780 -80.802640   
1  Hillsborough County  Florida  United States         33613 -82.436298   
2  Hillsborough County  Florida  United States         33613 -82

Eliminamos la palabra county

In [27]:
# Eliminamos la palabra 'County' de las columnas 'direccion' y 'ciudad'
data['direccion'] = data['direccion'].str.replace(r'\bCounty\b', '', regex=True).str.strip()
data['ciudad'] = data['ciudad'].str.replace(r'\bCounty\b', '', regex=True).str.strip()
print(data[['direccion', 'ciudad']].head())

# Guardamos el resultado 
processed_file_path = 'H:/Nueva carpeta/overpass-turbo.eu/Parques_de_diversiones_Centros_comerciales_2024.csv'
data.to_csv(processed_file_path, index=False)

                                 direccion        ciudad
0                Narvaez Drive, Titusville       Brevard
1          North 20th Street, Hillsborough  Hillsborough
2       East Fletcher Avenue, Hillsborough  Hillsborough
3  Hartford Street South, Saint Petersburg      Pinellas
4   North Dale Mabry Highway, Hillsborough  Hillsborough


Creamos las columnas id nombre y id ciudad

In [28]:
# Cargamos las rutas de los archivos
input_path = r"H:/Nueva carpeta/overpass-turbo.eu/Parques_de_diversiones_Centros_comerciales_2024.csv"
output_path = r"H:/Nueva carpeta/overpass-turbo.eu/Parques_de_diversiones_Centros_comerciales_2024_final.csv"
nombres_dim_path = "H:/Nueva carpeta/overpass-turbo.eu/nombres_dim.csv"
ciudades_dim_path = "H:/Nueva carpeta/overpass-turbo.eu/ciudades_dim.csv"

# Cargamos los catálogos
def load_catalog(path, key, value):
    catalog = {}
    if os.path.exists(path):
        with open(path, mode='r', encoding='utf-8') as f:
            reader = csv.DictReader(f)
            for row in reader:
                catalog[row[key]] = int(row[value])
    return catalog

# Guardamos los catálogos de ciudad y nombres
def save_catalog(path, catalog, key, value):
    with open(path, mode='w', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=[value, key])
        writer.writeheader()
        for k, v in catalog.items():
            writer.writerow({value: v, key: k})

# Generamos IDs únicos
def get_id(value, catalog):
    if value not in catalog:
        catalog[value] = len(catalog) + 1
    return catalog[value]

# Cargamos catálogos existentes
nombres_dict = load_catalog(nombres_dim_path, "nombre", "id_nombre")
ciudades_dict = load_catalog(ciudades_dim_path, "ciudad", "id_ciudad")

# Procesamos archivo de entrada
def process_file(input_path, output_path):
    if not os.path.exists(input_path):
        raise FileNotFoundError(f"El archivo de entrada no existe: {input_path}")

    with open(input_path, mode='r', encoding='utf-8') as infile:
        reader = csv.DictReader(infile)
        original_fieldnames = reader.fieldnames

        # Validamos las columnas columnas requeridas
        required_cols = {"nombre", "direccion", "latitud", "longitud", "ciudad", "estado", "pais", "codigo_postal"}
        missing_cols = required_cols - set(original_fieldnames)
        if missing_cols:
            raise ValueError(f"El CSV no contiene las siguientes columnas requeridas: {', '.join(missing_cols)}")

        # Nombres de las columnas
        fieldnames = [
            "id_nombre", "nombre", "direccion", "latitud", "longitud",
            "id_ciudad", "ciudad", "estado", "pais", "codigo_postal"
        ]

        # ordenamos y creamos el archivo de salida
        with open(output_path, mode='w', newline='', encoding='utf-8') as outfile:
            writer = csv.DictWriter(outfile, fieldnames=fieldnames)
            writer.writeheader()

            for row in reader:
                id_nombre = get_id(row["nombre"], nombres_dict)
                id_ciudad = get_id(row["ciudad"], ciudades_dict)

                new_row = {
                    "id_nombre": id_nombre,
                    "nombre": row["nombre"],
                    "direccion": row["direccion"],
                    "latitud": row["latitud"],
                    "longitud": row["longitud"],
                    "id_ciudad": id_ciudad,
                    "ciudad": row["ciudad"],
                    "estado": row["estado"],
                    "pais": row["pais"],
                    "codigo_postal": row["codigo_postal"]
                }
                writer.writerow(new_row)

process_file(input_path, output_path)

# Guardamos el catálogos actualizados 
save_catalog(nombres_dim_path, nombres_dict, "nombre", "id_nombre")
save_catalog(ciudades_dim_path, ciudades_dict, "ciudad", "id_ciudad")

print("Proceso completado con éxito.")

Proceso completado con éxito.


Pasamos el codigo a formato .parquet

In [30]:
# Cargamos el archivo CSV 
csv_path = r"H:/Nueva carpeta/overpass-turbo.eu/Parques_de_diversiones_Centros_comerciales_2024_final.csv"

# Ruta del archivo Parquet de salida
parquet_path = r"H:\Nueva carpeta\overpass-turbo.eu\Parques_de_diversiones_Centros_comerciales_2024_final.parquet"

df = pd.read_csv(csv_path)

# Guardamos el archivo Parquet
df.to_parquet(parquet_path, engine='pyarrow', index=False)

print(f"Archivo convertido exitosamente a Parquet: {parquet_path}")

Archivo convertido exitosamente a Parquet: H:\Nueva carpeta\overpass-turbo.eu\Parques_de_diversiones_Centros_comerciales_2024_final.parquet
